# Notebook d’analyse

## 0) Préambule

---

Ce notebook a pour objectif de réaliser une analyse complète d’un univers d’investissement multi-actifs à partir du fichier `analyse_donnees.xlsx`.

L’analyse suit la structure suivante :

1. **Tableau de bord** — Vue d’ensemble (performance, risque, structure sectorielle)
2. **Analyse de la performance** — Distribution et moteurs de rendement
3. **Analyse du risque** — Risque réalisé, implicite et structurel
4. **Valorisation et attentes de marché** — Multiples, croissance et consensus
5. **Analyse extra-financière (ESG)** — Facteurs de durabilité et impact sur la performance
6. **Analyse sectorielle** — Décomposition sectorielle
6. **Portefeuille** — Construction d'un portefeuille ESG qui maximise le ratio de sharpe (Ex Ante)

### 🎯 Objectif du notebook
Ce notebook a pour but de :

- Charger et **nettoyer les données** sources (Excel)  
- Calculer les **indicateurs clés** et préparer les visualisations  
- Générer les **graphiques Plotly** pour chaque partie de l’analyse  

### ⚙️ Structure technique
Chaque partie du notebook est divisée en **cellules identifiées** :
- **Markdown** → pour les titres et les explications  
- **Code Python** → pour les calculs et visualisations  
- **Sortie Plotly** → graphiques interactifs prêts à être exportés dans Streamlit  

### 0.1) Initialisation des librairies

In [1]:
# Manipulation et calcul
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Visualisation interactive
import plotly.express as px
import plotly.graph_objects as go

# Affichage et système
import openpyxl
import os
from datetime import datetime

# Options pandas pour un affichage propre
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Informations de contexte
print("✅ Librairies chargées avec succès")
print("Date d'exécution :", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

✅ Librairies chargées avec succès
Date d'exécution : 2025-11-13 21:48:27


### 0.2) Importation et nettoyage du fichier Excel

In [2]:
# Lecture de la feuille principale ('Feuil2')
df = pd.read_excel("analyse données.xlsx", sheet_name="Feuil2")

# Renommer la première colonne si elle est anonyme
if "Unnamed: 0" in df.columns:
    df.rename(columns={"Unnamed: 0": "Ticker"}, inplace=True)

# Nettoyage basique
df = df.dropna(how="all")                                    # Suppression des lignes complètement vides
df = df.drop_duplicates(subset="Ticker", keep="first")        # Suppression des doublons potentiels
df = df.replace([np.inf, -np.inf], np.nan)                    # Remplacement des infinis par NaN
df = df.dropna(axis=0, how="any", subset=["1 Year Total Return - Previous"])  # Garder les titres avec rendement connu

# --- Séparation indice / actions ---
# La première ligne correspond à l'indice global
indice = df.iloc[[0]].copy()
actions = df.iloc[1:].copy().reset_index(drop=True)

# Remplacer "#N/A Field Not Applicable" par NaN
indice = indice.map(lambda x: np.nan if isinstance(x, str) and "#N/A" in x else x)

# Aperçu des deux dataframes
print("\nExtrait de la ligne 'Indice' :")
display(indice)

print("\nAperçu du DataFrame 'Actions' :")
display(actions.head(5))


Extrait de la ligne 'Indice' :


,Ticker,1 Year Total Return - Previous,Volatility 360 Day Calc,BEst P/E Ratio,12 Month Put Implied Volatility,Price / Earnings - 5 Year Average,BEst LTG EPS,Upside with Target Price from Analyst,ESG Score,Sector (1),Sector (2),Risk Country
0,SX5E Index,9.68,16.20,15.57,16.30,NaN,8.33,0.08,NaN,NaN,NaN,NaN



Aperçu du DataFrame 'Actions' :


,Ticker,1 Year Total Return - Previous,Volatility 360 Day Calc,BEst P/E Ratio,12 Month Put Implied Volatility,Price / Earnings - 5 Year Average,BEst LTG EPS,Upside with Target Price from Analyst,ESG Score,Sector (1),Sector (2),Risk Country
0,TTE FP Equity,-17.37,20.98,8.23,19.98,NaN,9.17,0.18,1.37,Energy,Energy,FRANCE
1,RACE IM Equity,17.00,27.42,49.86,25.97,52.43,8.89,-0.01,0.11,Consumer Discretionary,Automobiles & Components,ITALY
2,BAS GY Equity,-5.82,27.68,13.64,25.11,NaN,2.75,0.17,-0.01,Materials,Materials,GERMANY
3,DHL GY Equity,3.05,27.02,12.88,25.01,12.99,9.35,0.12,0.88,Industrials,Transportation,GERMANY
4,ALV GY Equity,37.86,17.93,12.65,18.10,12.11,10.34,0.01,1.69,Financials,Insurance,GERMANY


### 0.3) Définition du style global (charte graphique Plotly)

In [3]:
# Palette de couleur Amundi
COLORS = {
    "primary": "rgb(8, 27, 72)",      # Bleu foncé - corporate
    "secondary": "rgb(81, 156, 221)", # Bleu clair - accent
    "background": "rgb(255, 255, 255)" # Fond blanc
}

# Définition du template global pour Plotly
PLOTLY_TEMPLATE = "plotly_white"

# Application des paramètres généraux
def apply_plotly_style(fig, title=""):
    """Applique la charte graphique globale à une figure Plotly."""
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(family="Noto Sans, Arial", size=20, color=COLORS["primary"]),
            x=0.02, xanchor="left"
        ),
        font=dict(family="Noto Sans, Arial", size=13, color=COLORS["primary"]),
        paper_bgcolor=COLORS["background"],
        plot_bgcolor=COLORS["background"],
        template=PLOTLY_TEMPLATE,
        xaxis=dict(showgrid=True, gridcolor="rgba(0,0,0,0.05)", mirror=True),
        yaxis=dict(showgrid=True, gridcolor="rgba(0,0,0,0.05)", mirror=True)
    )
    return fig

print("🎨 Charte graphique Plotly appliquée avec succès")

🎨 Charte graphique Plotly appliquée avec succès


## 1) Tableau de bord

---

### 1.1) KPI Cards (table compacte)

In [4]:
# Univers de travail (si tu ajoutes des filtres plus tard, remplace par 'fdf')
universe = actions

# Colonnes
COL_RET = "1 Year Total Return - Previous"       # Rendement 1 an (%)
COL_VOL = "Volatility 360 Day Calc"              # Volatilité 1 an (%)
COL_ESG = "ESG Score"                            # ESG (score)
COL_UP  = "Upside with Target Price from Analyst"  # Upside (à convertir en %)

# Normalisation Upside -> %
if COL_UP in universe.columns and universe[COL_UP].notna().any():
    p95 = np.nanpercentile(pd.to_numeric(universe[COL_UP], errors="coerce"), 95)
    if p95 <= 5:  # si les valeurs semblent en décimal, on multiplie par 100
        universe[COL_UP] = pd.to_numeric(universe[COL_UP], errors="coerce") * 100.0

# Helpers robustes
def nanmean(s):   return float(np.nanmean(pd.to_numeric(s, errors="coerce")))
def nanmedian(s): return float(np.nanmedian(pd.to_numeric(s, errors="coerce")))

# Calculs
ret_mean,  ret_med  = nanmean(universe[COL_RET]), nanmedian(universe[COL_RET])
vol_mean,  vol_med  = nanmean(universe[COL_VOL]), nanmedian(universe[COL_VOL])
esg_mean,  esg_med  = (nanmean(universe[COL_ESG]) if COL_ESG in universe else np.nan,
                       nanmedian(universe[COL_ESG]) if COL_ESG in universe else np.nan)
sharpe_series = (pd.to_numeric(universe[COL_RET], errors="coerce") /
                 pd.to_numeric(universe[COL_VOL], errors="coerce")).replace([np.inf, -np.inf], np.nan)
sharpe_mean, sharpe_med = float(np.nanmean(sharpe_series)), float(np.nanmedian(sharpe_series))

# Table KPI (Indicateur | Moyenne | Médiane)
kpi_table = pd.DataFrame({
    "Indicateur": ["Rendement 1 an", "Volatilité 1 an", "Score ESG", "Sharpe Proxy"],
    "Moyenne":    [f"{ret_mean:.2f} %", f"{vol_mean:.2f} %",
                   (f"{esg_mean:.2f}" if not np.isnan(esg_mean) else "—"),
                   f"{sharpe_mean:.2f}"],
    "Médiane":    [f"{ret_med:.2f} %", f"{vol_med:.2f} %",
                   (f"{esg_med:.2f}" if not np.isnan(esg_med) else "—"),
                   f"{sharpe_med:.2f}"],
})

# Table Plotly (charte visuelle compacte)
fig_kpi = go.Figure(data=[go.Table(
    header=dict(
        values=["<b>Indicateur</b>", "<b>Moyenne</b>", "<b>Médiane</b>"],
        fill_color=COLORS["primary"],
        font=dict(color="white", family="Noto Sans, Arial"),
        align="left",
        height=30
    ),
    cells=dict(
        values=[kpi_table[c] for c in ["Indicateur", "Moyenne", "Médiane"]],
        fill_color=COLORS["background"],
        font=dict(color=COLORS["primary"], family="Noto Sans, Arial", size=13),
        align="left",
        height=28
    )
)])
fig_kpi = apply_plotly_style(fig_kpi, title="KPI - Vue d'ensemble (univers SX5E)")
fig_kpi.update_layout(margin=dict(l=0, r=0, t=50, b=5), height=200)
fig_kpi.show()


### 1.2) Scatter Risk / Return

In [5]:

fdf = actions.copy()

fig_risk_return = px.scatter(
    fdf,
    x="Volatility 360 Day Calc",
    y="1 Year Total Return - Previous",
    color="Sector (1)",
    hover_data={
        "Ticker": True,
        "Sector (1)": True,
        "Risk Country": True,
        "1 Year Total Return - Previous": ":.2f",
        "Volatility 360 Day Calc": ":.2f",
        "Upside with Target Price from Analyst": ":.2f" if "Upside with Target Price from Analyst" in fdf else False
    },
    template="plotly_white",
    color_discrete_sequence=px.colors.sequential.Blues,  # palette cohérente dérivée du bleu clair
)

# Lignes de médianes
x_med = float(fdf["Volatility 360 Day Calc"].median())
y_med = float(fdf["1 Year Total Return - Previous"].median())
fig_risk_return.add_hline(y=y_med, line_dash="dot", line_color="rgba(0,0,0,0.25)")
fig_risk_return.add_vline(x=x_med, line_dash="dot", line_color="rgba(0,0,0,0.25)")

# Style & labels
fig_risk_return.update_traces(marker=dict(size=9, line=dict(width=0)))
fig_risk_return.update_layout(
    legend_title_text="Secteur",
    xaxis_title="Volatilité 360j (%)",
    yaxis_title="Rendement 1 an (%)",
    annotations=[
        dict(
            text=f"n = {len(fdf)} actifs",
            xref="paper", yref="paper", x=0.99, y=0.01,
            xanchor="right", yanchor="bottom",
            showarrow=False,
            font=dict(color="rgba(0,0,0,0.6)", size=12, family="Noto Sans, Arial")
        )
    ]
)
fig_risk_return = apply_plotly_style(fig_risk_return, title="Carte Risque / Rendement")
fig_risk_return.show()


### 1.3) Donut répartition par secteur

In [6]:
# Univers (si filtres plus tard, remplace 'actions' par 'fdf')
universe = actions.copy()

# Préparation des données
sect_col = "Sector (1)"
tmp = (
    universe
    .assign(**{sect_col: universe[sect_col].fillna("Inconnu")})
    .groupby(sect_col, as_index=False)
    .agg(n_titles=("Ticker", "count"))
    .sort_values("n_titles", ascending=False)
)

# Option : regrouper les plus petits secteurs dans "Autres" si trop nombreux
TOP_N = 12
if len(tmp) > TOP_N:
    top = tmp.head(TOP_N - 1)
    other = pd.DataFrame({
        sect_col: ["Autres"],
        "n_titles": [tmp.iloc[TOP_N-1:]["n_titles"].sum()]
    })
    tmp = pd.concat([top, other], ignore_index=True)

# Figure donut
fig_sector_donut = px.pie(
    tmp,
    names=sect_col,
    values="n_titles",
    hole=0.6,
    color=sect_col,
    color_discrete_sequence=px.colors.sequential.Blues[::-1],  # nuances cohérentes
)

# Texte et hover propres
fig_sector_donut.update_traces(
    textposition="inside",
    textinfo="label+percent",
    hovertemplate="<b>%{label}</b><br>Nombre d'actifs: %{value}<br>Part: %{percent}<extra></extra>"
)

# Style & layout
fig_sector_donut = apply_plotly_style(fig_sector_donut, title="Répartition par secteur (taille = nombre d'actifs)")
fig_sector_donut.update_layout(
    showlegend=False,                 # la légende est redondante avec le texte interne
    margin=dict(l=0, r=0, t=50, b=5),
    height=420
)

fig_sector_donut.show()

### 1.4) Bar Top 5 / Bottom 5 (1 an)

In [7]:
fdf = actions.copy()

# Tri
top5 = fdf.nlargest(5, "1 Year Total Return - Previous").copy()
top5["Groupe"] = "Top 5"
bottom5 = fdf.nsmallest(5, "1 Year Total Return - Previous").copy()
bottom5["Groupe"] = "Bottom 5"

tb = pd.concat([top5, bottom5], ignore_index=True)

# Ordre pour un bar horizontal lisible
tb = tb.sort_values(by=["Groupe", "1 Year Total Return - Previous"], ascending=[True, True])

# Couleurs par groupe
color_map = {"Top 5": COLORS["secondary"], "Bottom 5": COLORS["primary"]}
tb["Color"] = tb["Groupe"].map(color_map)

fig_top_bottom = go.Figure()

for grp in ["Top 5", "Bottom 5"]:
    sub = tb[tb["Groupe"] == grp]
    fig_top_bottom.add_trace(go.Bar(
        x=sub["1 Year Total Return - Previous"],
        y=sub["Ticker"],
        orientation="h",
        name=grp,
        marker_color=color_map[grp],
        hovertemplate=(
            "<b>%{y}</b><br>" +
            "Secteur: %{customdata[0]}<br>" +
            "Pays: %{customdata[1]}<br>" +
            "Rendement 1Y: %{x:.2f}%<extra></extra>"
        ),
        customdata=np.stack([sub["Sector (1)"], sub["Risk Country"]], axis=-1)
    ))

fig_top_bottom.update_layout(
    barmode="relative",
    xaxis_title="Rendement 1 an (%)",
    yaxis_title="Ticker",
    legend_title_text="Groupe",

)
fig_top_bottom = apply_plotly_style(fig_top_bottom, title="Top 5 / Bottom 5 - Rendement 1 an")
fig_top_bottom.show()


### 1.5) Tableau synthétique 5 titres à regarder

In [8]:
fdf = actions.copy()

# Colonnes utiles & calculs auxiliaires
col_pe   = "BEst P/E Ratio"
col_pe5  = "Price / Earnings - 5 Year Average"
col_esg  = "ESG Score"
col_up   = "Upside with Target Price from Analyst"
col_vol  = "Volatility 360 Day Calc"
col_ret  = "1 Year Total Return - Previous"

fdf["Sharpe_proxy"] = (fdf[col_ret] / fdf[col_vol]).replace([np.inf, -np.inf], np.nan)
if col_pe in fdf and col_pe5 in fdf:
    fdf["PE_premium_5Y_%"] = (fdf[col_pe] - fdf[col_pe5]) / fdf[col_pe5] * 100

# Sélection 5 catégories (dédupliquées)
shortlist = []

def add_best(df, sort_col, ascending, label):
    cand = df.sort_values(sort_col, ascending=ascending)
    for _, row in cand.iterrows():
        if row["Ticker"] not in [s["Ticker"] for s in shortlist]:
            shortlist.append({
                "Raison": label,
                "Ticker": row["Ticker"],
                "Sector (1)": row.get("Sector (1)", None),
                "Risk Country": row.get("Risk Country", None),
                "Return 1Y (%)": row.get(col_ret, np.nan),
                "Vol 360j (%)": row.get(col_vol, np.nan),
                "Sharpe proxy": row.get("Sharpe_proxy", np.nan),
                "ESG": row.get(col_esg, np.nan),
                "P/E": row.get(col_pe, np.nan),
                "P/E 5Y": row.get(col_pe5, np.nan),
                "Décote P/E vs 5Y (%)": row.get("PE_premium_5Y_%", np.nan),
                "Upside (%)": row.get(col_up, np.nan),
            })
            break

# 1. Meilleur Rendement 1Y
add_best(fdf.dropna(subset=[col_ret]), col_ret, False, "Meilleur rendement 1Y")

# 2. Meilleur Sharpe proxy
add_best(fdf.dropna(subset=["Sharpe_proxy"]), "Sharpe_proxy", False, "Meilleur rendement/risque")

# 3. Meilleur ESG
if col_esg in fdf:
    add_best(fdf.dropna(subset=[col_esg]), col_esg, False, "Meilleur score ESG")

# 4. Plus forte DÉCOTE P/E vs 5Y (valeur la plus négative)
if "PE_premium_5Y_%" in fdf:
    add_best(fdf.dropna(subset=["PE_premium_5Y_%"]), "PE_premium_5Y_%", True, "Plus forte décote P/E vs 5Y")

# 5. Plus fort Upside
if col_up in fdf:
    add_best(fdf.dropna(subset=[col_up]), col_up, False, "Plus fort Upside analystes")

summary5_df = pd.DataFrame(shortlist)

# Mise en forme des valeurs numériques
for c in ["Return 1Y (%)", "Vol 360j (%)", "Upside (%)", "Décote P/E vs 5Y (%)"]:
    if c in summary5_df:
        summary5_df[c] = summary5_df[c].map(lambda x: f"{x:.2f}%" if pd.notna(x) else "—")
for c in ["Sharpe proxy", "ESG", "P/E", "P/E 5Y"]:
    if c in summary5_df:
        summary5_df[c] = summary5_df[c].map(lambda x: f"{x:.2f}" if pd.notna(x) else "—")

# Table Plotly
fig_summary5 = go.Figure(data=[go.Table(
    header=dict(
        values=[f"<b>{h}</b>" for h in ["Raison","Ticker","Sector","Risk Country","Return 1Y (%)","Vol 360j (%)","Sharpe proxy","ESG","P/E","P/E 5Y","Décote P/E vs 5Y (%)","Upside (%)"]],
        fill_color=COLORS["primary"],
        font=dict(color="white", family="Noto Sans, Arial"),
        align="left",
        height=28
    ),
    cells=dict(
        values=[summary5_df.get(h, ["—"]*len(summary5_df)) for h in ["Raison","Ticker","Sector (1)","Risk Country","Return 1Y (%)","Vol 360j (%)","Sharpe proxy","ESG","P/E","P/E 5Y","Décote P/E vs 5Y (%)","Upside (%)"]],
        fill_color=COLORS["background"],
        font=dict(color=COLORS["primary"], family="Noto Sans, Arial", size=13),
        align="left",
        height=26
    )
)])
fig_summary5 = apply_plotly_style(fig_summary5, title="Synthèse - 5 titres à regarder")
fig_summary5.update_layout(margin=dict(
    l=0, r=0, t=50, b=0),
    height=360)
fig_summary5.show()


## 2) Analyse de performance

---

### 2.1) Distribution des rendements 1 an

In [9]:
universe = actions.copy()
col_ret = "1 Year Total Return - Previous"

s = universe[col_ret].astype(float).dropna()
ret_min, ret_max, ret_mean, ret_med = float(s.min()), float(s.max()), float(s.mean()), float(s.median())

# Histogramme plus léger et plus fin
fig_perf_dist = px.histogram(
    universe,
    x=col_ret,
    nbins=35,  # plus de bacs → barres plus fines
    template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]],
)
fig_perf_dist.update_traces(
    marker_line_width=0,
    opacity=0.85
)

# Lignes de moyenne / médiane
fig_perf_dist.add_vline(x=ret_med, line_dash="dot", line_color="rgba(0,0,0,0.35)")
fig_perf_dist.add_vline(x=ret_mean, line_dash="dash", line_color=COLORS["primary"])

# Style & mise en page
fig_perf_dist = apply_plotly_style(fig_perf_dist, title="Distribution des rendements 1 an")
fig_perf_dist.update_layout(
    xaxis_title="Rendement 1 an (%)",
    yaxis_title="Effectif",
    margin=dict(l=0, r=0, t=70, b=40),  # ↑ espace haut pour le titre / ↑ bas pour le padding
    height=420,
    bargap=0.15,  # plus d'espace entre les barres
    annotations=[
        dict(
            text=f"Min {ret_min:.2f}%  |  Moy {ret_mean:.2f}%  |  Med {ret_med:.2f}%  |  Max {ret_max:.2f}%",
            xref="paper", yref="paper", x=0.0, y=1.10, showarrow=False,
            font=dict(family="Noto Sans, Arial", size=12, color="rgba(0,0,0,0.7)")
        )
    ]
)

fig_perf_dist.show()


### 2.2) Box plot - rendements par secteur

In [10]:
universe = actions.copy()
col_ret, col_sector = "1 Year Total Return - Previous", "Sector (1)"
universe[col_sector] = universe[col_sector].fillna("Inconnu")

fig_box_sector = px.box(
    universe, x=col_sector, y=col_ret, points=False,
    template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]],
)
fig_box_sector.update_layout(
    xaxis_title="Secteur",
    yaxis_title="Rendement 1 an (%)",
    margin=dict(l=0, r=0, t=50, b=120),  # b plus grand pour lisibilité des labels x
    height=520
)
fig_box_sector = apply_plotly_style(fig_box_sector, title="Dispersion des rendements par secteur")
fig_box_sector.show()

### 2.3) Bar chart rendement moyen par pays

In [11]:
universe = actions.copy()
col_ret, col_country = "1 Year Total Return - Previous", "Risk Country"
universe[col_country] = universe[col_country].fillna("Inconnu")

agg = (
    universe.groupby(col_country, as_index=False)
    .agg(ret_mean=(col_ret, "mean"), ret_median=(col_ret, "median"), n=("Ticker", "count"))
    .sort_values("ret_mean", ascending=False)
)

fig_country_bar = px.bar(
    agg, x=col_country, y="ret_mean",
    template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]],
    hover_data={
        col_country: False, "ret_mean":":.2f", "ret_median":":.2f", "n": True
    }
)
fig_country_bar.update_traces(
    hovertemplate="<b>%{x}</b><br>Moy: %{y:.2f}%<br>Méd: %{customdata[1]:.2f}%<br>n: %{customdata[2]}<extra></extra>"
)
fig_country_bar.update_layout(
    xaxis_title="Pays (Risk Country)",
    yaxis_title="Rendement moyen 1 an (%)",
    margin=dict(l=0, r=0, t=50, b=80),
    height=520
)
fig_country_bar = apply_plotly_style(fig_country_bar, title="Rendement moyen par pays")
fig_country_bar.show()


### 2.4) Rendement moyen par décile de volatilité

In [12]:
universe = actions.copy()
col_ret, col_vol = "1 Year Total Return - Previous", "Volatility 360 Day Calc"

# Données valides
dfq = universe[[col_ret, col_vol]].dropna().copy()

# Crée 10 déciles ordonnés (garde un ordre stable même si certains bins sont vides)
labels = [f"D{i}" for i in range(1, 11)]
dfq["Vol_Decile"] = pd.qcut(
    dfq[col_vol].rank(method="first"),
    q=10,
    labels=labels,
    duplicates="drop"  # par sécurité si beaucoup d'égalités
)
# Force la catégorie ordonnée (important pour l'affichage/tri)
dfq["Vol_Decile"] = pd.Categorical(dfq["Vol_Decile"], categories=labels, ordered=True)

# Agrégats — observed=True pour supprimer le FutureWarning et ignorer les catégories non observées
by_dec = (
    dfq.groupby("Vol_Decile", observed=True, as_index=False)
       .agg(ret_mean=(col_ret, "mean"), n=(col_ret, "count"))
       .sort_values("Vol_Decile")
)

# Bar chart
fig_ret_by_decile = px.bar(
    by_dec, x="Vol_Decile", y="ret_mean",
    template="plotly_white",
    color_discrete_sequence=[COLORS["primary"]],
    hover_data={"ret_mean":":.2f", "n": True}
)

fig_ret_by_decile.update_layout(
    xaxis_title="Déciles de volatilité (D1 = plus faible)",
    yaxis_title="Rendement moyen 1 an (%)",
    margin=dict(l=0, r=0, t=50, b=5),
    height=420
)
fig_ret_by_decile = apply_plotly_style(fig_ret_by_decile, title="Rémunération du risque - rendement moyen par décile de volatilité")
fig_ret_by_decile.show()


### 2.5) Scatter - Rendement vs P/E (value vs growth)

In [13]:
universe   = actions.copy()
col_ret    = "1 Year Total Return - Previous"
col_pe     = "BEst P/E Ratio"
col_sector = "Sector (1)"

# Données valides
plot_df = universe[[col_ret, col_pe, col_sector, "Ticker", "Risk Country"]].dropna().copy()
plot_df[col_sector] = plot_df[col_sector].fillna("Inconnu")

# Arrays numériques
x = pd.to_numeric(plot_df[col_pe], errors="coerce").to_numpy()
y = pd.to_numeric(plot_df[col_ret], errors="coerce").to_numpy()
mask = np.isfinite(x) & np.isfinite(y)
xv, yv = x[mask], y[mask]

# Régression linéaire simple (pente/interception) et corrélation R
if len(xv) >= 2:
    slope, intercept = np.polyfit(xv, yv, 1)
    R = float(np.corrcoef(xv, yv)[0, 1])
else:
    slope, intercept, R = 0.0, float(np.nanmean(yv) if len(yv) else 0.0), np.nan

# Figure scatter
fig_ret_vs_pe = px.scatter(
    plot_df,
    x=col_pe,
    y=col_ret,
    color=col_sector,
    hover_data={
        "Ticker": True,
        "Risk Country": True,
        col_sector: True,
        col_pe:":.2f",
        col_ret:":.2f",
    },
    template="plotly_white",
    color_discrete_sequence=px.colors.sequential.Blues
)
fig_ret_vs_pe.update_traces(marker=dict(size=9, opacity=0.85, line=dict(width=0)))

# Droite de tendance ajoutée manuellement
if len(xv) >= 2:
    x_line = np.linspace(xv.min(), xv.max(), 100)
    y_line = slope * x_line + intercept
    fig_ret_vs_pe.add_trace(
        go.Scatter(
            x=x_line, y=y_line,
            mode="lines",
            name="Trend",
            line=dict(color=COLORS["primary"], width=2, dash="dash"),
            hoverinfo="skip",
            showlegend=False
        )
    )

# Mise en forme & titre enrichi
trend_direction = "négative" if slope < 0 else "positive"
r_text = f"(R = {R:.2f}, pente {trend_direction})" if np.isfinite(R) else ""
title = f"Rendement 1 an vs P/E - lecture value/growth {r_text}"

fig_ret_vs_pe.update_layout(
    xaxis_title="P/E (BEst)",
    yaxis_title="Rendement 1 an (%)",
    legend_title_text="Secteur",
    margin=dict(l=0, r=0, t=70, b=40),
    height=480
)
fig_ret_vs_pe = apply_plotly_style(fig_ret_vs_pe, title=title)

fig_ret_vs_pe.show()


## 3.0) Analyse de risque

---

### 3.1) Vol implicite vs Vol historique

In [14]:
universe = actions.copy()
col_vol = "Volatility 360 Day Calc"
col_iv  = "12 Month Put Implied Volatility"

plot_df = universe[[col_vol, col_iv, "Ticker", "Sector (1)", "Risk Country"]].dropna().copy()
plot_df["Prime_IV_Vol"] = plot_df[col_iv] - plot_df[col_vol]

fig_iv_vs_vol = px.scatter(
    plot_df,
    x=col_vol, y=col_iv,
    color="Sector (1)",
    hover_data={
        "Ticker": True,
        "Risk Country": True,
        "Sector (1)": True,
        col_vol:":.2f", col_iv:":.2f",
        "Prime_IV_Vol":":.2f"
    },
    template="plotly_white",
    color_discrete_sequence=px.colors.sequential.Blues
)
fig_iv_vs_vol.update_traces(marker=dict(size=9, opacity=0.9, line=dict(width=0)))

# Diagonale y = x
xy_min = float(np.nanmin([plot_df[col_vol].min(), plot_df[col_iv].min()]))
xy_max = float(np.nanmax([plot_df[col_vol].max(), plot_df[col_iv].max()]))
fig_iv_vs_vol.add_trace(go.Scatter(
    x=[xy_min, xy_max], y=[xy_min, xy_max],
    mode="lines", line=dict(color="rgba(0,0,0,0.25)", dash="dot"), name="y = x",
    hoverinfo="skip", showlegend=False
))

fig_iv_vs_vol.update_layout(
    xaxis_title="Volatilité 1 an (%)",
    yaxis_title="Vol implicite 12m (%)",
    legend_title_text="Secteur",
    margin=dict(l=0, r=0, t=60, b=40),
    height=480
)
fig_iv_vs_vol = apply_plotly_style(fig_iv_vs_vol, title="Vol implicite vs Vol historique - prime de volatilité (IV − Vol)")
fig_iv_vs_vol.show()


### 3.2) Prime de volatilité - Top / Bottom 10

In [15]:
universe = actions.copy()
col_vol = "Volatility 360 Day Calc"
col_iv  = "12 Month Put Implied Volatility"

tb = (
    universe[[ "Ticker", "Sector (1)", "Risk Country", col_vol, col_iv ]]
    .dropna()
    .assign(Prime=lambda d: d[col_iv] - d[col_vol])
)

top10 = tb.nlargest(10, "Prime").copy()
bot10 = tb.nsmallest(10, "Prime").copy()
top10["Groupe"] = "Top 10 prime (IV−Vol)"
bot10["Groupe"] = "Bottom 10 prime (IV−Vol)"
tbl = pd.concat([top10, bot10], ignore_index=True)

# Mise en forme
for c in [col_vol, col_iv, "Prime"]:
    tbl[c] = tbl[c].map(lambda x: f"{x:.2f}%")

fig_premium_tbl = go.Figure(data=[go.Table(
    header=dict(
        values=[ "<b>Groupe</b>", "<b>Ticker</b>", "<b>Secteur</b>", "<b>Pays</b>", "<b>Vol 1 an</b>", "<b>IV 12m</b>", "<b>Prime (IV−Vol)</b>" ],
        fill_color=COLORS["primary"], font=dict(color="white", family="Noto Sans, Arial"), align="left", height=30
    ),
    cells=dict(
        values=[ tbl["Groupe"], tbl["Ticker"], tbl["Sector (1)"], tbl["Risk Country"], tbl[col_vol], tbl[col_iv], tbl["Prime"] ],
        fill_color=COLORS["background"], font=dict(color=COLORS["primary"], family="Noto Sans, Arial", size=13),
        align="left", height=26
    )
)])
fig_premium_tbl = apply_plotly_style(fig_premium_tbl, title="Prime de volatilité - extrêmes (Top/Bottom 10)")
fig_premium_tbl.update_layout(margin=dict(l=0, r=0, t=60, b=5), height=420)
fig_premium_tbl.show()


### 3.3 Distribution des volatilités (1 an)

In [16]:
universe = actions.copy()
col_vol = "Volatility 360 Day Calc"
s = universe[col_vol].astype(float).dropna()

fig_vol_dist = px.histogram(
    universe, x=col_vol, nbins=30, template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]]
)
fig_vol_dist.update_traces(marker_line_width=0, opacity=0.85)
fig_vol_dist.add_vline(x=float(s.median()), line_dash="dot", line_color="rgba(0,0,0,0.35)")
fig_vol_dist.add_vline(x=float(s.mean()),   line_dash="dash", line_color=COLORS["primary"])

fig_vol_dist.update_layout(
    xaxis_title="Volatilité 1 an (%)",
    yaxis_title="Effectif",
    margin=dict(l=0, r=0, t=60, b=40),
    height=420
)
fig_vol_dist = apply_plotly_style(fig_vol_dist, title="Distribution des volatilités (1 an)")
fig_vol_dist.show()


### 3.4) Bêta (proxy) par secteur + top/bottom

In [17]:
col_vol = "Volatility 360 Day Calc"

# Volatilité de l'indice (df 'indice' = 1 ligne)
vol_index = pd.to_numeric(indice[col_vol].squeeze(), errors="coerce")
vol_index = float(vol_index) if np.isfinite(vol_index) else np.nan

# Calcul du bêta proxy individuel
beta_df = actions[["Ticker", "Sector (1)", "Risk Country", col_vol]].dropna().copy()
beta_df["beta_proxy"] = (
    beta_df[col_vol] / vol_index if np.isfinite(vol_index) else np.nan
)

# Agrégation par secteur — médiane plutôt que moyenne
agg_beta = (
    beta_df.groupby("Sector (1)", as_index=False)
           .agg(beta_median=("beta_proxy", "median"), n=("Ticker", "count"))
           .sort_values("beta_median", ascending=False)
)

# Bar chart — bêta médian par secteur
fig_beta_sector = px.bar(
    agg_beta,
    x="Sector (1)", 
    y="beta_median",
    template="plotly_white",
    color_discrete_sequence=[COLORS["primary"]],
    hover_data={"beta_median":":.2f","n":True}
)

# Mise en forme
fig_beta_sector.update_layout(
    xaxis_title="Secteur",
    yaxis_title="β (proxy = Vol titre / Vol indice, médiane sectorielle)",
    margin=dict(l=0, r=0, t=60, b=120),
    height=520
)

fig_beta_sector = apply_plotly_style(fig_beta_sector, title="Bêta (proxy) médian par secteur")
fig_beta_sector.show()

# Table Top/Bottom 10 β titres
tb = pd.concat([
    beta_df.nlargest(10, "beta_proxy").assign(Groupe="Top 10 β"),
    beta_df.nsmallest(10, "beta_proxy").assign(Groupe="Bottom 10 β")
], ignore_index=True)

tb["beta_proxy"] = tb["beta_proxy"].map(lambda x: f"{x:.2f}")
fig_beta_tbl = go.Figure(data=[go.Table(
    header=dict(
        values=["<b>Groupe</b>", "<b>Ticker</b>", "<b>Secteur</b>", "<b>Pays</b>", "<b>β (proxy)</b>"],
        fill_color=COLORS["primary"], font=dict(color="white", family="Noto Sans, Arial"),
        align="left", height=30
    ),
    cells=dict(
        values=[tb["Groupe"], tb["Ticker"], tb["Sector (1)"], tb["Risk Country"], tb["beta_proxy"]],
        fill_color=COLORS["background"], font=dict(color=COLORS["primary"], family="Noto Sans, Arial", size=13),
        align="left", height=26
    )
)])
fig_beta_tbl = apply_plotly_style(fig_beta_tbl, title="β (proxy) extrêmes par titre")
fig_beta_tbl.update_layout(margin=dict(l=0, r=0, t=60, b=5), height=420)
fig_beta_tbl.show()

### 3.5 Frontière efficiente (simplifiée)

In [18]:
col_ret = "1 Year Total Return - Previous"
col_vol = "Volatility 360 Day Calc"

df = actions[[col_ret, col_vol, "Ticker", "Sector (1)"]].dropna().copy()
df.rename(columns={col_ret:"ret", col_vol:"vol"}, inplace=True)

# Convex hull (enveloppe supérieure) — algo monotone chain simplifié
pts = df[["vol","ret"]].to_numpy()
order = np.argsort(pts[:,0])
pts = pts[order]

def upper_hull(points):
    hull = []
    def cross(o, a, b):
        return (a[0]-o[0])*(b[1]-o[1]) - (a[1]-o[1])*(b[0]-o[0])
    for p in points:
        while len(hull) >= 2 and cross(hull[-2], hull[-1], p) >= 0:
            hull.pop()
        hull.append(tuple(p))
    # enlever segments décroissants en ret (garder l'enveloppe “efficiente”)
    hull = [hull[0]] + [p for i,p in enumerate(hull[1:-1], start=1) if p[1]>=hull[i-1][1]] + [hull[-1]]
    return np.array(hull)

frontier = upper_hull(pts)
# filtre points non NaN et strictement croissants en vol
frontier = frontier[np.argsort(frontier[:,0])]

# Figure
fig_frontier = go.Figure()

# Nuage
fig_frontier.add_trace(go.Scatter(
    x=df["vol"], y=df["ret"], mode="markers",
    marker=dict(size=8, color="rgba(81,156,221,0.7)"),
    name="Titres", text=df["Ticker"],
    hovertemplate="<b>%{text}</b><br>Vol: %{x:.2f}%<br>Rend: %{y:.2f}%<extra></extra>"
))

# Frontière
fig_frontier.add_trace(go.Scatter(
    x=frontier[:,0], y=frontier[:,1], mode="lines+markers",
    line=dict(color=COLORS["primary"], width=2),
    marker=dict(size=6, color=COLORS["primary"]),
    name="Frontière (simplifiée)", hoverinfo="skip"
))

fig_frontier.update_layout(
    xaxis_title="Volatilité 1 an (%)",
    yaxis_title="Rendement 1 an (%)",
    margin=dict(l=0, r=0, t=60, b=40),
    height=500,
    legend_title_text=""
)
fig_frontier = apply_plotly_style(fig_frontier, title="Frontière efficiente (nuage titres + enveloppe supérieure)")
fig_frontier.show()


### 3.6) Heatmap de corrélation (risque & facteurs)

In [19]:
cols = {
    "Rendement 1 an (%)":          "1 Year Total Return - Previous",
    "Volatilité 1 an (%)":         "Volatility 360 Day Calc",
    "IV 12m (%)":                  "12 Month Put Implied Volatility",
    "Upside (%)":                  "Upside with Target Price from Analyst",
    "ESG (score)":                 "ESG Score",
    "P/E (BEst)":                  "BEst P/E Ratio",
    "LTG EPS (%)":                 "BEst LTG EPS"
}

dfc = actions[[c for c in cols.values() if c in actions.columns]].copy()

# Upside en % si besoin
if "Upside with Target Price from Analyst" in dfc.columns:
    p95 = np.nanpercentile(pd.to_numeric(dfc["Upside with Target Price from Analyst"], errors="coerce"), 95)
    if p95 <= 5:
        dfc["Upside with Target Price from Analyst"] = pd.to_numeric(dfc["Upside with Target Price from Analyst"], errors="coerce") * 100.0

# Renommer pour la lisibilité
rename_map = {v:k for k,v in cols.items() if v in dfc.columns}
dfc = dfc.rename(columns=rename_map)

# Conversion numérique et corrélation
dfc = dfc.apply(pd.to_numeric, errors="coerce")
corr = dfc.corr(method="pearson").round(2)

fig_corr = px.imshow(
    corr, text_auto=True, aspect="auto", color_continuous_scale="Blues",
    template="plotly_white"
)
fig_corr.update_layout(
    coloraxis_colorbar=dict(title="ρ"),
    margin=dict(l=0, r=0, t=60, b=5),
    height=520
)
fig_corr = apply_plotly_style(fig_corr, title="Corrélations - risque & facteurs")
fig_corr.show()


## 4.0) Valorisation & attentes

---

### 4.1) P/E vs LTG EPS scatter + tendance

In [20]:
PE   = "BEst P/E Ratio"
LTG  = "BEst LTG EPS"
SECT = "Sector (1)"

df = actions[[PE, LTG, SECT, "Ticker", "Risk Country"]].dropna().copy()
df[SECT] = df[SECT].fillna("Inconnu")

# Régression linéaire (sans statsmodels)
x = pd.to_numeric(df[LTG], errors="coerce").to_numpy()
y = pd.to_numeric(df[PE],  errors="coerce").to_numpy()
mask = np.isfinite(x) & np.isfinite(y)
xv, yv = x[mask], y[mask]
slope, intercept = (np.polyfit(xv, yv, 1) if len(xv) >= 2 else (0.0, float(np.nanmean(yv)) if len(yv) else 0.0))
R = float(np.corrcoef(xv, yv)[0, 1]) if len(xv) >= 2 else np.nan

fig_pe_ltg = px.scatter(
    df, x=LTG, y=PE, color=SECT,
    hover_data={"Ticker":True, "Risk Country":True, SECT:True, LTG:":.2f", PE:":.2f"},
    template="plotly_white", color_discrete_sequence=px.colors.sequential.Blues
)
fig_pe_ltg.update_traces(marker=dict(size=9, opacity=0.9, line=dict(width=0)))

# Ajout de la droite de tendance
if len(xv) >= 2:
    x_line = np.linspace(xv.min(), xv.max(), 100)
    y_line = slope * x_line + intercept
    fig_pe_ltg.add_trace(go.Scatter(
        x=x_line, y=y_line, mode="lines",
        line=dict(color=COLORS["primary"], width=2, dash="dash"),
        hoverinfo="skip", showlegend=False
    ))

# Mise en forme
r_text = f"(R = {R:.2f})" if np.isfinite(R) else ""
fig_pe_ltg.update_layout(
    xaxis_title="LTG EPS (croissance attendue, %)",
    yaxis_title="P/E (BEst)",
    legend_title_text="Secteur",
    margin=dict(l=0, r=0, t=70, b=40), height=480
)
fig_pe_ltg = apply_plotly_style(fig_pe_ltg, title=f"P/E vs LTG EPS - valorisation vs croissance {r_text}")
fig_pe_ltg.show()


### 4.2) Prime/Décote P/E vs moyenne 5 ans

In [21]:
PE   = "BEst P/E Ratio"
PE5  = "Price / Earnings - 5 Year Average"

df = actions[["Ticker", "Sector (1)", "Risk Country", PE, PE5]].dropna().copy()
df["PE_premium_5Y_%"] = (pd.to_numeric(df[PE], errors="coerce") - pd.to_numeric(df[PE5], errors="coerce")) \
                        / pd.to_numeric(df[PE5], errors="coerce") * 100.0

# Sélection Top 10 primes et Top 10 décotes
top_premium = df.nlargest(10, "PE_premium_5Y_%").assign(Groupe="Prime (Top10)")
top_discount = df.nsmallest(10, "PE_premium_5Y_%").assign(Groupe="Décote (Top10)")
plot_df = pd.concat([top_premium, top_discount], ignore_index=True)
plot_df = plot_df.sort_values(["Groupe", "PE_premium_5Y_%"], ascending=[False, False])

# Figure
fig_pe_premium = go.Figure()
for grp, color in [("Prime (Top10)", COLORS["secondary"]), ("Décote (Top10)", COLORS["primary"])]:
    sub = plot_df[plot_df["Groupe"] == grp]
    fig_pe_premium.add_trace(go.Bar(
        x=sub["Ticker"], y=sub["PE_premium_5Y_%"], name=grp,
        marker_color=color,
        hovertemplate="<b>%{x}</b><br>Prime/Décote: %{y:.2f}%<br>PE: %{customdata[0]:.2f} | PE 5Y: %{customdata[1]:.2f}<extra></extra>",
        customdata=np.stack([sub[PE], sub[PE5]], axis=-1)
    ))

fig_pe_premium.update_layout(
    xaxis_title="Ticker", yaxis_title="Prime/Décote vs 5 ans (%)",
    barmode="group", margin=dict(l=0, r=0, t=60, b=120), height=520,
    legend_title_text=""
)
fig_pe_premium = apply_plotly_style(fig_pe_premium, title="P/E actuel vs moyenne 5 ans - primes et décotes (Top 20)")
fig_pe_premium.show()


### 4.3) P/E par secteur — box plot

In [22]:
PE, SECT = "BEst P/E Ratio", "Sector (1)"
df = actions[[PE, SECT]].dropna().copy()
df[SECT] = df[SECT].fillna("Inconnu")

fig_pe_box = px.box(
    df, x=SECT, y=PE, points=False, template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]]
)
fig_pe_box.update_layout(
    xaxis_title="Secteur", yaxis_title="P/E (BEst)",
    margin=dict(l=0, r=0, t=60, b=120), height=520
)
fig_pe_box = apply_plotly_style(fig_pe_box, title="Distribution des P/E par secteur")
fig_pe_box.show()

### 4.4) PEG ratio Top/Bottom + distribution

In [23]:
PE, LTG = "BEst P/E Ratio", "BEst LTG EPS"

df = actions[["Ticker", "Sector (1)", "Risk Country", PE, LTG]].dropna().copy()
# Évite divisions par zéro ou LTG négatif (option : garder mais l'annoter)
df["PEG"] = pd.to_numeric(df[PE], errors="coerce") / pd.to_numeric(df[LTG], errors="coerce")
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["PEG"])

# Table Top/Bottom 10
tb = pd.concat([
    df.nsmallest(10, "PEG").assign(Groupe="Top 10 PEG (bas)"),
    df.nlargest(10, "PEG").assign(Groupe="Top 10 PEG (haut)")
], ignore_index=True)

tb_disp = tb.copy()
for c in [PE, LTG, "PEG"]:
    tb_disp[c] = tb_disp[c].map(lambda v: f"{v:.2f}" if pd.notna(v) else "—")

fig_peg_tbl = go.Figure(data=[go.Table(
    header=dict(
        values=["<b>Groupe</b>","<b>Ticker</b>","<b>Secteur</b>","<b>Pays</b>","<b>P/E</b>","<b>LTG EPS</b>","<b>PEG</b>"],
        fill_color=COLORS["primary"], font=dict(color="white", family="Noto Sans, Arial"),
        align="left", height=30
    ),
    cells=dict(
        values=[tb_disp["Groupe"], tb_disp["Ticker"], tb_disp["Sector (1)"], tb_disp["Risk Country"], tb_disp[PE], tb_disp[LTG], tb_disp["PEG"]],
        fill_color=COLORS["background"], font=dict(color=COLORS["primary"], family="Noto Sans, Arial", size=13),
        align="left", height=26
    )
)])
fig_peg_tbl = apply_plotly_style(fig_peg_tbl, title="PEG ratio extrêmes (Top/Bottom 10)")
fig_peg_tbl.update_layout(margin=dict(l=0, r=0, t=60, b=5), height=420)
fig_peg_tbl.show()

# Distribution des PEG
# Distribution des PEG — version robuste et lisible
# Règle : on garde seulement LTG > 0 et on clippe aux [p1, p99] pour éviter les valeurs polluantes

df_dist = df.copy()

# 1) Restreindre aux cas interprétables : LTG > 0
df_dist = df_dist[pd.to_numeric(df_dist[LTG], errors="coerce") > 0].copy()

# 2) Supprimer inf/NaN, puis calculer des bornes robustes
df_dist = df_dist.replace([np.inf, -np.inf], np.nan).dropna(subset=["PEG"])
p1, p99 = np.nanpercentile(df_dist["PEG"], [1, 99])

# 3) Clipper (filtrer) aux percentiles pour une lecture propre
mask_clip = (df_dist["PEG"] >= p1) & (df_dist["PEG"] <= p99)
df_plot = df_dist[mask_clip].copy()

# Comptes pour transparence
excluded_total = len(df) - len(df_plot)           # par rapport au df de départ de la section PEG
excluded_reason = {
    "LTG <= 0 ou NaN": int((pd.to_numeric(df[LTG], errors="coerce") <= 0).sum()),
    "PEG hors [p1, p99]": int(len(df_dist) - len(df_plot))
}

# Histogramme propre
fig_peg_hist = px.histogram(
    df_plot, x="PEG", nbins=30, template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]]
)
fig_peg_hist.update_traces(marker_line_width=0, opacity=0.85)

# Lignes de moyenne/médiane (sur l’échantillon filtré)
fig_peg_hist.add_vline(x=float(df_plot["PEG"].median()), line_dash="dot", line_color="rgba(0,0,0,0.35)")
fig_peg_hist.add_vline(x=float(df_plot["PEG"].mean()),   line_dash="dash", line_color=COLORS["primary"])

# Mise en page & annotation de transparence
fig_peg_hist.update_layout(
    xaxis_title="PEG (LTG > 0, clip [p1, p99])",
    yaxis_title="Effectif",
    margin=dict(l=0, r=0, t=70, b=40),
    height=420,
    bargap=0.15,
    annotations=[
        dict(
            text=f"Exclus: total {excluded_total} | LTG≤0/NaN: {excluded_reason['LTG <= 0 ou NaN']} | Hors [p1,p99]: {excluded_reason['PEG hors [p1, p99]']}",
            xref="paper", yref="paper", x=0.0, y=1.10, showarrow=False,
            font=dict(family='Noto Sans, Arial', size=12, color='rgba(0,0,0,0.7)')
        )
    ]
)
fig_peg_hist = apply_plotly_style(fig_peg_hist, title="Distribution du PEG ratio")
fig_peg_hist.show()



### 4.5) Upside potentiel par secteur (médian)

In [24]:
UPS, SECT = "Upside with Target Price from Analyst", "Sector (1)"
df = actions[[UPS, SECT]].dropna().copy()
df[SECT] = df[SECT].fillna("Inconnu")

# Normalise en % si nécessaire (idempotent)
p95 = np.nanpercentile(pd.to_numeric(df[UPS], errors="coerce"), 95)
if p95 <= 5:
    df[UPS] = pd.to_numeric(df[UPS], errors="coerce") * 100.0

agg = df.groupby(SECT, as_index=False).agg(up_median=(UPS, "median"), n=(UPS, "count")).sort_values("up_median", ascending=False)

fig_up_sector = px.bar(
    agg, x=SECT, y="up_median",
    template="plotly_white",
    color_discrete_sequence=[COLORS["primary"]],
    hover_data={"up_median":":.2f","n":True}
)
fig_up_sector.update_layout(
    xaxis_title="Secteur", yaxis_title="Upside médian (%)",
    margin=dict(l=0, r=0, t=60, b=120), height=520
)
fig_up_sector = apply_plotly_style(fig_up_sector, title="Upside potentiel (médian) par secteur")
fig_up_sector.show()

### 4.6) Upside vs P/E scatter + tendance

In [25]:
PE, UPS, SECT = "BEst P/E Ratio", "Upside with Target Price from Analyst", "Sector (1)"

df = actions[[PE, UPS, SECT, "Ticker", "Risk Country"]].dropna().copy()
df[SECT] = df[SECT].fillna("Inconnu")

# Upside en % si besoin (idempotent)
p95 = np.nanpercentile(pd.to_numeric(df[UPS], errors="coerce"), 95)
if p95 <= 5:
    df[UPS] = pd.to_numeric(df[UPS], errors="coerce") * 100.0

# Régression (sans statsmodels)
x = pd.to_numeric(df[PE], errors="coerce").to_numpy()
y = pd.to_numeric(df[UPS], errors="coerce").to_numpy()
mask = np.isfinite(x) & np.isfinite(y)
xv, yv = x[mask], y[mask]
slope, intercept = (np.polyfit(xv, yv, 1) if len(xv) >= 2 else (0.0, float(np.nanmean(yv)) if len(yv) else 0.0))
R = float(np.corrcoef(xv, yv)[0, 1]) if len(xv) >= 2 else np.nan

fig_up_pe = px.scatter(
    df, x=PE, y=UPS, color=SECT,
    hover_data={"Ticker":True, "Risk Country":True, SECT:True, PE:":.2f", UPS:":.2f"},
    template="plotly_white",
    color_discrete_sequence=px.colors.sequential.Blues
)
fig_up_pe.update_traces(marker=dict(size=9, opacity=0.9, line=dict(width=0)))

if len(xv) >= 2:
    x_line = np.linspace(xv.min(), xv.max(), 100)
    y_line = slope * x_line + intercept
    fig_up_pe.add_trace(go.Scatter(
        x=x_line, y=y_line, mode="lines",
        line=dict(color=COLORS["primary"], width=2, dash="dash"),
        hoverinfo="skip", showlegend=False
    ))

r_text = f"(R = {R:.2f})" if np.isfinite(R) else ""
fig_up_pe.update_layout(
    xaxis_title="P/E (BEst)", yaxis_title="Upside analystes (%)",
    legend_title_text="Secteur",
    margin=dict(l=0, r=0, t=70, b=40), height=480
)
fig_up_pe = apply_plotly_style(fig_up_pe, title=f"Upside analystes vs P/E - décote vs attentes {r_text}")
fig_up_pe.show()


## 5.0) Analyse extra-financière (ESG)

---

### 5.1) Distribution des scores ESG

In [26]:
col_esg = "ESG Score"
df = actions[[col_esg]].dropna().copy()
s = pd.to_numeric(df[col_esg], errors="coerce")

fig_esg_dist = px.histogram(
    df, 
    x=col_esg, 
    nbins=40,  # plus de bins = barres plus fines
    template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]]
)

# Ajustements visuels
fig_esg_dist.update_traces(marker_line_width=0, opacity=0.85)

# Ajout de moyenne et médiane
fig_esg_dist.add_vline(
    x=float(s.mean()), 
    line_dash="dash", 
    line_color=COLORS["primary"]
)
fig_esg_dist.add_vline(
    x=float(s.median()), 
    line_dash="dot", 
    line_color="rgba(0,0,0,0.35)"
)

# Mise en forme
fig_esg_dist.update_layout(
    xaxis_title="Score ESG",
    yaxis_title="Effectif",
    margin=dict(l=0, r=0, t=60, b=50),  # un peu plus d’espace bas
    height=420,
    bargap=0.25  # espacement entre barres pour effet plus fin
)

fig_esg_dist = apply_plotly_style(fig_esg_dist, title="Distribution des scores ESG (barres affinées)")
fig_esg_dist.show()

### 5.2) ESG par secteur (médiane)

In [27]:
col_esg, col_sector = "ESG Score", "Sector (1)"
df = actions[[col_esg, col_sector]].dropna().copy()
df[col_sector] = df[col_sector].fillna("Inconnu")

agg = df.groupby(col_sector, as_index=False).agg(
    esg_median=(col_esg, "median"),
    esg_mean=(col_esg, "mean"),
    n=(col_esg, "count")
).sort_values("esg_median", ascending=False)

fig_esg_sector = px.bar(
    agg, x=col_sector, y="esg_median",
    template="plotly_white",
    color_discrete_sequence=[COLORS["primary"]],
    hover_data={"esg_mean":":.2f","n":True}
)
fig_esg_sector.update_layout(
    xaxis_title="Secteur",
    yaxis_title="Score ESG médian",
    margin=dict(l=0, r=0, t=60, b=120),
    height=520
)
fig_esg_sector = apply_plotly_style(fig_esg_sector, title="Score ESG médian par secteur")
fig_esg_sector.show()


### 5.3) ESG vs Rendement 1 an

In [28]:
col_esg, col_ret, col_sector = "ESG Score", "1 Year Total Return - Previous", "Sector (1)"
df = actions[[col_esg, col_ret, col_sector, "Ticker"]].dropna().copy()
df[col_sector] = df[col_sector].fillna("Inconnu")

x = pd.to_numeric(df[col_esg], errors="coerce").to_numpy()
y = pd.to_numeric(df[col_ret], errors="coerce").to_numpy()
mask = np.isfinite(x) & np.isfinite(y)
xv, yv = x[mask], y[mask]
slope, intercept = (np.polyfit(xv, yv, 1) if len(xv) >= 2 else (0.0, np.nanmean(yv)))
R = float(np.corrcoef(xv, yv)[0, 1]) if len(xv) >= 2 else np.nan

fig_esg_ret = px.scatter(
    df, x=col_esg, y=col_ret, color=col_sector,
    hover_data={"Ticker":True, col_esg:":.2f", col_ret:":.2f"},
    template="plotly_white", color_discrete_sequence=px.colors.sequential.Blues
)
fig_esg_ret.update_traces(marker=dict(size=9, opacity=0.85))

# Ligne de tendance
if len(xv) >= 2:
    x_line = np.linspace(xv.min(), xv.max(), 100)
    y_line = slope * x_line + intercept
    fig_esg_ret.add_trace(go.Scatter(
        x=x_line, y=y_line, mode="lines",
        line=dict(color=COLORS["primary"], width=2, dash="dash"),
        hoverinfo="skip", showlegend=False
    ))

r_text = f"(R = {R:.2f})" if np.isfinite(R) else ""
fig_esg_ret.update_layout(
    xaxis_title="Score ESG",
    yaxis_title="Rendement 1 an (%)",
    margin=dict(l=0, r=0, t=70, b=40),
    height=480
)
fig_esg_ret = apply_plotly_style(fig_esg_ret, title=f"ESG vs performance financière {r_text}")
fig_esg_ret.show()

### 5.4 ESG vs Volatilité

In [29]:
col_esg, col_vol, col_sector = "ESG Score", "Volatility 360 Day Calc", "Sector (1)"
df = actions[[col_esg, col_vol, col_sector, "Ticker"]].dropna().copy()
df[col_sector] = df[col_sector].fillna("Inconnu")

x = pd.to_numeric(df[col_esg], errors="coerce").to_numpy()
y = pd.to_numeric(df[col_vol], errors="coerce").to_numpy()
mask = np.isfinite(x) & np.isfinite(y)
xv, yv = x[mask], y[mask]
slope, intercept = (np.polyfit(xv, yv, 1) if len(xv) >= 2 else (0.0, np.nanmean(yv)))
R = float(np.corrcoef(xv, yv)[0, 1]) if len(xv) >= 2 else np.nan

fig_esg_vol = px.scatter(
    df, x=col_esg, y=col_vol, color=col_sector,
    hover_data={"Ticker":True, col_esg:":.2f", col_vol:":.2f"},
    template="plotly_white", color_discrete_sequence=px.colors.sequential.Blues
)
fig_esg_vol.update_traces(marker=dict(size=9, opacity=0.85))

if len(xv) >= 2:
    x_line = np.linspace(xv.min(), xv.max(), 100)
    y_line = slope * x_line + intercept
    fig_esg_vol.add_trace(go.Scatter(
        x=x_line, y=y_line, mode="lines",
        line=dict(color=COLORS["primary"], width=2, dash="dash"),
        hoverinfo="skip", showlegend=False
    ))

r_text = f"(R = {R:.2f})" if np.isfinite(R) else ""
fig_esg_vol.update_layout(
    xaxis_title="Score ESG",
    yaxis_title="Volatilité 1 an (%)",
    margin=dict(l=0, r=0, t=70, b=40),
    height=480
)
fig_esg_vol = apply_plotly_style(fig_esg_vol, title=f"ESG vs risque (volatilité) {r_text}")
fig_esg_vol.show()

### 5.5) ESG vs Upside - matrice catégorielle

In [30]:
col_esg = "ESG Score"
col_up  = "Upside with Target Price from Analyst"

df = actions[[col_esg, col_up, "Sector (1)"]].dropna().copy()
df["Sector (1)"] = df["Sector (1)"].fillna("Inconnu")

# Normalisation de Upside en %
p95 = np.nanpercentile(pd.to_numeric(df[col_up], errors="coerce"), 95)
if p95 <= 5:
    df[col_up] = pd.to_numeric(df[col_up], errors="coerce") * 100.0

# Déciles ESG & Upside
df["ESG_bin"] = pd.qcut(
    pd.to_numeric(df[col_esg], errors="coerce"), 
    5, 
    labels=["Très bas","Bas","Moyen","Haut","Très haut"]
)
df["Upside_bin"] = pd.qcut(
    pd.to_numeric(df[col_up], errors="coerce"), 
    5, 
    labels=["Très faible","Faible","Modéré","Élevé","Très élevé"]
)

# ✅ Correction du FutureWarning : on ajoute observed=True
heat = df.groupby(["ESG_bin", "Upside_bin"], as_index=False, observed=True).size()

# Heatmap
fig_esg_up = px.density_heatmap(
    heat, 
    x="ESG_bin", 
    y="Upside_bin", 
    z="size",
    color_continuous_scale="Blues", 
    text_auto=True,
    template="plotly_white"
)

fig_esg_up.update_layout(
    xaxis_title="Score ESG (quantiles)",
    yaxis_title="Upside analystes (quantiles)",
    coloraxis_colorbar=dict(title="Nombre d'actifs"),
    margin=dict(l=0, r=0, t=60, b=60),
    height=500
)
fig_esg_up = apply_plotly_style(
    fig_esg_up, 
    title="Matrice ESG vs Upside - lecture combinée valorisation / durabilité"
)
fig_esg_up.show()


## 6.0) Analyse sectorielle

---

### 6.1) Tableau KPI sectoriel

In [31]:
SECT = "Sector (1)"
RET  = "1 Year Total Return - Previous"
VOL  = "Volatility 360 Day Calc"
ESG  = "ESG Score"
UP   = "Upside with Target Price from Analyst"
PE   = "BEst P/E Ratio"
LTG  = "BEst LTG EPS"

df = actions.copy()
df[SECT] = df[SECT].fillna("Inconnu")

# Upside en % si nécessaire
if UP in df.columns and df[UP].notna().any():
    p95 = np.nanpercentile(pd.to_numeric(df[UP], errors="coerce"), 95)
    if p95 <= 5:
        df[UP] = pd.to_numeric(df[UP], errors="coerce") * 100.0

# PEG ratio si P/E et LTG existent
if PE in df.columns and LTG in df.columns:
    pe_num  = pd.to_numeric(df[PE], errors="coerce")
    ltg_num = pd.to_numeric(df[LTG], errors="coerce")
    df["PEG"] = np.where(ltg_num > 0, pe_num / ltg_num, np.nan)
else:
    df["PEG"] = np.nan

# Sharpe proxy
ret_num = pd.to_numeric(df[RET], errors="coerce")
vol_num = pd.to_numeric(df[VOL], errors="coerce")
df["sharpe_proxy"] = (ret_num / vol_num).replace([np.inf, -np.inf], np.nan)

# Agrégats
agg = (
    df.groupby(SECT, as_index=False)
      .agg(n=("Ticker","count"),
           ret_med=(RET,"median"),
           vol_med=(VOL,"median"),
           sharpe_med=("sharpe_proxy","median"))
)

# Ajouts dynamiques
def merge_median(source_col, out_col, base):
    if source_col in df.columns:
        m = df.groupby(SECT, as_index=False)[source_col].median().rename(columns={source_col: out_col})
        return base.merge(m, on=SECT, how="left")
    return base

agg = merge_median(ESG, "esg_med", agg)
agg = merge_median(UP,  "up_med",  agg)
agg = merge_median(PE,  "pe_med",  agg)
agg = merge_median("PEG", "peg_med", agg)

# Mise en forme
tbl = agg.sort_values("n", ascending=False).copy()
fmt_pct = lambda x: (f"{x:.2f} %" if pd.notna(x) else "—")
fmt_num = lambda x: (f"{x:.2f}" if pd.notna(x) else "—")

for c in ["ret_med", "vol_med", "up_med"]:
    if c in tbl.columns: tbl[c] = tbl[c].map(fmt_pct)
for c in ["sharpe_med", "esg_med", "pe_med", "peg_med"]:
    if c in tbl.columns: tbl[c] = tbl[c].map(fmt_num)

tbl["n"] = tbl["n"].astype(int)

# Colonnes à afficher (ordre fixe + filtre dispo)
cols_display = [SECT, "n", "ret_med", "vol_med", "sharpe_med", "esg_med", "up_med", "pe_med", "peg_med"]
cols_display = [c for c in cols_display if c in tbl.columns]

# En-têtes alignés 1:1 avec les colonnes
headers_map = {
    SECT: "Secteur",
    "n": "n",
    "ret_med": "Rend. méd.",
    "vol_med": "Vol méd.",
    "sharpe_med": "Sharpe méd.",
    "esg_med": "ESG méd.",
    "up_med": "Upside méd.",
    "pe_med": "P/E méd.",
    "peg_med": "PEG méd."
}
headers = [f"<b>{headers_map[c]}</b>" for c in cols_display]

# Tableau Plotly proprement aligné
fig_sector_kpi = go.Figure(data=[go.Table(
    header=dict(
        values=headers,
        fill_color=COLORS["primary"],
        font=dict(color="white", family="Noto Sans, Arial"),
        align="left",
        height=30
    ),
    cells=dict(
        values=[tbl[c] for c in cols_display],
        fill_color=COLORS["background"],
        font=dict(color=COLORS["primary"], family="Noto Sans, Arial", size=13),
        align="left",
        height=26
    )
)])

fig_sector_kpi = apply_plotly_style(fig_sector_kpi, title="KPI sectoriels - médianes et effectifs")
fig_sector_kpi.update_layout(margin=dict(l=0, r=0, t=60, b=5), height=460)
fig_sector_kpi.show()


### 6.2) Carte Risque/Rendement - centroïdes sectoriels

In [32]:
SECT, RET, VOL = "Sector (1)", "1 Year Total Return - Previous", "Volatility 360 Day Calc"
df = actions[[SECT, RET, VOL]].dropna().copy()
df[SECT] = df[SECT].fillna("Inconnu")

centroids = (
    df.groupby(SECT, as_index=False)
      .agg(ret_med=(RET, "median"), vol_med=(VOL, "median"), n=("Sector (1)","count"))
)

fig_sector_rr = px.scatter(
    centroids,
    x="vol_med", y="ret_med",
    size="n", size_max=40,
    color_discrete_sequence=[COLORS["secondary"]],
    template="plotly_white",
    hover_data={"n":True, "vol_med":":.2f", "ret_med":":.2f"},
    text=SECT
)
fig_sector_rr.update_traces(textposition="top center", marker=dict(line=dict(width=0)))
fig_sector_rr.update_layout(
    xaxis_title="Volatilité 1 an (médiane, %)",
    yaxis_title="Rendement 1 an (médiane, %)",
    margin=dict(l=0, r=0, t=60, b=40),
    height=480, showlegend=False
)
fig_sector_rr = apply_plotly_style(fig_sector_rr, title="Carte Risque / Rendement — centroïdes sectoriels")
fig_sector_rr.show()


### 6.3) Barres groupées - Return & Vol (médian)

In [33]:
SECT, RET, VOL = "Sector (1)", "1 Year Total Return - Previous", "Volatility 360 Day Calc"
df = actions[[SECT, RET, VOL]].dropna().copy()
df[SECT] = df[SECT].fillna("Inconnu")
agg = df.groupby(SECT, as_index=False).agg(ret_med=(RET,"median"), vol_med=(VOL,"median")).sort_values("ret_med", ascending=False)

fig_group = go.Figure()
fig_group.add_bar(x=agg[SECT], y=agg["ret_med"], name="Rendement médian 1 an", marker_color=COLORS["secondary"])
fig_group.add_bar(x=agg[SECT], y=agg["vol_med"], name="Volatilité médiane 1 an", marker_color=COLORS["primary"])
fig_group.update_layout(
    barmode="group",
    xaxis_title="Secteur",
    yaxis_title="%",
    margin=dict(l=0, r=0, t=60, b=120),
    height=520
)
fig_group = apply_plotly_style(fig_group, title="Return vs Vol (médian) — comparaison sectorielle")
fig_group.show()

### 6.4) Prime/Décote P/E vs 5 ans - médiane sectorielle

In [34]:
SECT, PE, PE5 = "Sector (1)", "BEst P/E Ratio", "Price / Earnings - 5 Year Average"
df = actions[[SECT, PE, PE5]].dropna().copy()
df[SECT] = df[SECT].fillna("Inconnu")
df["PE_premium_5Y_%"] = (pd.to_numeric(df[PE], errors="coerce") - pd.to_numeric(df[PE5], errors="coerce")) \
                        / pd.to_numeric(df[PE5], errors="coerce") * 100.0

agg = df.groupby(SECT, as_index=False).agg(premium_med=("PE_premium_5Y_%","median"), n=(PE,"count")).sort_values("premium_med")
fig_prem = px.bar(
    agg, x=SECT, y="premium_med",
    template="plotly_white",
    color_discrete_sequence=[COLORS["primary"]],
    hover_data={"premium_med":":.2f","n":True}
)
fig_prem.update_layout(
    xaxis_title="Secteur",
    yaxis_title="Prime/Décote P/E vs 5 ans (médiane, %)",
    margin=dict(l=0, r=0, t=60, b=120),
    height=520
)
fig_prem = apply_plotly_style(fig_prem, title="Prime/Décote P/E vs 5 ans médiane sectorielle")
fig_prem.show()


### 6.5) ESG médian vs Return médian - bulle sectorielle

In [35]:
SECT, ESG, RET = "Sector (1)", "ESG Score", "1 Year Total Return - Previous"

# Conserver les lignes où ESG et Return sont tous deux disponibles
df = actions[[SECT, ESG, RET]].copy()
df[SECT] = df[SECT].fillna("Inconnu")
df_valid = df.dropna(subset=[ESG, RET])

# Agrégats par secteur
agg = (
    df_valid.groupby(SECT, as_index=False)
            .agg(esg_med=(ESG, "median"), ret_med=(RET, "median"))
            .merge(
                df_valid.groupby(SECT).size().reset_index(name="n"),
                on=SECT, how="left"
            )
            .sort_values("ret_med", ascending=False)
)

# Scatter “bulle” sectoriel
fig_esg_ret_sec = px.scatter(
    agg, x="esg_med", y="ret_med",
    size="n", size_max=40,
    template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]],
    hover_data={"n": True, "esg_med":":.2f", "ret_med":":.2f"},
    text=SECT
)
fig_esg_ret_sec.update_traces(textposition="top center", marker=dict(line=dict(width=0)))
fig_esg_ret_sec.update_layout(
    xaxis_title="ESG (médian)",
    yaxis_title="Rendement 1 an (médian, %)",
    margin=dict(l=0, r=0, t=60, b=40),
    height=480,
    showlegend=False
)
fig_esg_ret_sec = apply_plotly_style(fig_esg_ret_sec, title="ESG (médian) vs Return (médian) par secteur")
fig_esg_ret_sec.show()


### 6.6) Radar profil sectoriel vs univers (z-scores)

In [36]:
# === Variables ===
SECT, RET, VOL, ESG, UP, PE = (
    "Sector (1)", "1 Year Total Return - Previous", "Volatility 360 Day Calc",
    "ESG Score", "Upside with Target Price from Analyst", "BEst P/E Ratio"
)

df = actions[[SECT, RET, VOL, ESG, UP, PE]].copy()
df[SECT] = df[SECT].fillna("Inconnu")

# Upside en % si nécessaire
if UP in df.columns and df[UP].notna().any():
    p95 = np.nanpercentile(pd.to_numeric(df[UP], errors="coerce"), 95)
    if p95 <= 5:
        df[UP] = pd.to_numeric(df[UP], errors="coerce") * 100.0

# === Calculs ===
metrics = {"Return 1Y (%)": RET, "Vol 1Y (%)": VOL, "ESG": ESG, "Upside (%)": UP, "P/E": PE}
sec_med = df.groupby(SECT, observed=True).median(numeric_only=True)[list(metrics.values())]

def zscore(series: pd.Series) -> pd.Series:
    mu = np.nanmedian(series)
    sigma = np.nanstd(series)
    if not np.isfinite(sigma) or sigma == 0:
        sigma = 1.0
    return (series - mu) / sigma

sec_z = sec_med.apply(zscore)
sec_z = sec_z.rename(columns={v: k for k, v in metrics.items()})

if sec_z.empty:
    print("Aucun secteur disponible pour le radar (données insuffisantes).")
else:
    cats = list(sec_z.columns)
    cats_closed = cats + [cats[0]]

    fig_radar = go.Figure()

    # Ligne de référence "Univers"
    fig_radar.add_trace(go.Scatterpolar(
        r=[0]*len(cats_closed), theta=cats_closed,
        name="Univers (réf=0)",
        line=dict(color="rgba(0,0,0,0.35)", dash="dot"),
        hoverinfo="skip", showlegend=True
    ))

    # Traces par secteur
    sectors = list(sec_z.index)
    for idx, sect in enumerate(sectors):
        row = sec_z.loc[sect]
        vals = row.values.tolist() + [row.values.tolist()[0]]
        fig_radar.add_trace(go.Scatterpolar(
            r=vals, theta=cats_closed, fill="toself",
            name=sect,
            line=dict(color="#081B48"),   # Bleu foncé de la charte
            fillcolor="rgba(8,27,72,0.3)", # Remplissage semi-transparent
            visible=(idx == 0)
        ))

    # === Menu déroulant repositionné ===
    buttons = []
    n_traces = 1 + len(sectors)
    for i, sect in enumerate(sectors):
        visible = [True] + [False]*len(sectors)
        visible[i+1] = True
        buttons.append(dict(
            label=sect,
            method="update",
            args=[
                {"visible": visible},
                {"title": f"Profil sectoriel (z-scores) — {sect} vs univers"}
            ],
        ))

    fig_radar.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, showgrid=True, gridcolor="rgba(0,0,0,0.06)")
        ),
        margin=dict(l=0, r=0, t=60, b=40),
        height=520,
        showlegend=True,
        updatemenus=[dict(
            type="dropdown",
            x=0.02, y=1.02,          # ↓ baisse du sélecteur sous le titre
            xanchor="left", yanchor="top",
            buttons=buttons,
            bgcolor="white",
            bordercolor="rgba(0,0,0,0.1)"
        )]
    )

    # === Titre stylisé ===
    initial_title = f"Profil sectoriel (z-scores) - {sectors[0]} vs univers"
    fig_radar = apply_plotly_style(fig_radar, title=initial_title)

    fig_radar.show()


## 7.0) Portefeuille ESG maximisant le ratio de sharpe (Ex Ante)

---

### 7.1) Filtre d'exclusion ESG sectoriel (>= médiane du secteur)

In [37]:
# Colonnes (adapter ici si besoin)
COL_SECTOR = "Sector (1)"
COL_RET1Y  = "1 Year Total Return - Previous"
COL_VOL    = "Volatility 360 Day Calc"
COL_PE     = "BEst P/E Ratio"
COL_LTG    = "BEst LTG EPS"
COL_PE5Y   = "Price / Earnings - 5 Year Average"
COL_UP     = "Upside with Target Price from Analyst"
COL_ESG    = "ESG Score"

# Hyperparamètres déterministes (ne pas modifier pour rester 100% quant)
WINSOR_P    = (2.5, 97.5)     # winsorisation intra-secteur
RIDGE_ALPHA = 1.0             # pénalité L2 pour la régression factorielle
N_TARGET    = 30              # taille cible (approx.) de l'univers candidat
EPS_MIN     = 1e-9            # poids min. technique pour "obligatoires"
LAMBDA_GRID = np.logspace(-3, 2, 50)  # grille pour Markowitz (max EU - l * Var)

# Utility: robust MAD (médiane des écarts absolus)
def _mad(x):
    x = pd.to_numeric(pd.Series(x), errors="coerce")
    med = np.nanmedian(x)
    mad = np.nanmedian(np.abs(x - med))
    return mad if mad and np.isfinite(mad) and mad > 0 else np.nanstd(x)

# Winsorisation par groupe (secteur)
def winsorize_by_group(df, group_col, cols, p_low=2.5, p_high=97.5):
    out = df.copy()
    for g, gdf in df.groupby(group_col):
        idx = gdf.index
        for c in cols:
            if c not in gdf.columns: 
                continue
            x = pd.to_numeric(gdf[c], errors="coerce")
            lo, hi = np.nanpercentile(x, [p_low, p_high]) if np.isfinite(x).any() else (np.nan, np.nan)
            if np.isnan(lo) or np.isnan(hi):
                continue
            out.loc[idx, c] = np.clip(x, lo, hi)
    return out

# Z-score par secteur (médiane/MAD ou std fallback)
def zscore_by_group(df, group_col, cols):
    out = df.copy()
    for c in cols:
        if c not in df.columns: 
            continue
        z = []
        for g, gdf in df.groupby(group_col):
            x = pd.to_numeric(gdf[c], errors="coerce")
            mu = np.nanmedian(x)
            s  = _mad(x)
            if not np.isfinite(s) or s == 0:
                s = np.nanstd(x)
                if not np.isfinite(s) or s == 0:
                    z.extend([np.nan]*len(gdf))
                    continue
            z.extend((x - mu)/s)
        out[f"Z::{c}"] = pd.Series(z, index=out.index)
    return out

# Normalisation "Upside": si petit (probablement en décimal), le mettre en %
def normalize_upside_inplace(df, col_up):
    if col_up in df.columns and df[col_up].notna().any():
        u = pd.to_numeric(df[col_up], errors="coerce")
        p95 = np.nanpercentile(u, 95)
        if np.isfinite(p95) and p95 <= 5:
            df[col_up] = u * 100.0

# Nettoyage général
def basic_clean(df):
    df = df.replace(["#N/A Field Not Applicable", "#N/A", "N/A", "NaN"], np.nan)
    df = df.dropna(how="all")
    return df


# === Filtre d'exclusion ESG sectoriel (>= médiane du secteur) ===
# Règle : on conserve uniquement les titres dont l'ESG >= médiane ESG de leur secteur.
# Garde-fou de faisabilité : si un secteur devient vide, on réintègre 1 titre (voir fallback ci-dessous).

import numpy as np
import pandas as pd

# Si 'work' n'existe pas encore, on part de 'actions'
if "work" not in globals():
    work = actions.copy()

# Vérifications minimales
if COL_ESG not in work.columns or COL_SECTOR not in work.columns:
    print("ℹ️ Filtre ESG non appliqué (colonnes ESG ou Sector manquantes).")
else:
    # Convertir ESG en numérique
    work[COL_ESG] = pd.to_numeric(work[COL_ESG], errors="coerce")
    work[COL_SECTOR] = work[COL_SECTOR].fillna("Inconnu")

    # Médiane ESG par secteur (ignore NaN)
    esg_med_by_sector = work.groupby(COL_SECTOR, observed=True)[COL_ESG].median()

    # Mapper la médiane au niveau ligne
    work["__ESG_med_sector"] = work[COL_SECTOR].map(esg_med_by_sector)

    # Masque cœur : ESG >= médiane du secteur
    mask_core = work[COL_ESG] >= work["__ESG_med_sector"]

    # Survivants initiaux après filtre strict
    survivors = work[mask_core].copy()

    # Garde-fou : garantir ≥ 1 titre par secteur (au niveau du dataset filtré)
    exceptions = []
    for sect, g in work.groupby(COL_SECTOR, observed=True):
        has_any = (survivors[COL_SECTOR] == sect).any()
        if not has_any:
            # 1) Si ESG non-NaN disponibles : reprendre le meilleur ESG
            g_valid_esg = g.dropna(subset=[COL_ESG]).sort_values(COL_ESG, ascending=False)
            if len(g_valid_esg) > 0:
                survivors = pd.concat([survivors, g_valid_esg.head(1)], axis=0)
                exceptions.append(f"{sect} (réintégré: meilleur ESG)")
            else:
                # 2) Fallback déterministe sans ESG utilisable :
                #    a) max Return 1Y si dispo
                #    b) sinon min Vol 1Y si dispo
                #    c) sinon première ligne
                picked = None
                if COL_RET1Y in g.columns:
                    g_num = g.dropna(subset=[COL_RET1Y])
                    if len(g_num) > 0:
                        g_num = g_num.copy()
                        g_num[COL_RET1Y] = pd.to_numeric(g_num[COL_RET1Y], errors="coerce")
                        picked = g_num.sort_values(COL_RET1Y, ascending=False).head(1)
                if picked is None and COL_VOL in g.columns:
                    g_num = g.dropna(subset=[COL_VOL])
                    if len(g_num) > 0:
                        g_num = g_num.copy()
                        g_num[COL_VOL] = pd.to_numeric(g_num[COL_VOL], errors="coerce")
                        picked = g_num.sort_values(COL_VOL, ascending=True).head(1)
                if picked is None:
                    picked = g.head(1)

                survivors = pd.concat([survivors, picked], axis=0)
                exceptions.append(f"{sect} (fallback sans ESG non-NaN)")

    # Nettoyage & message
    work = survivors.drop(columns=["__ESG_med_sector"], errors="ignore").copy()
    work.reset_index(drop=True, inplace=True)

    if len(exceptions) > 0:
        print("⚠️ Règle ESG appliquée : certains secteurs n’avaient plus de survivants après filtre.")
        print("   Réintégration automatique (1 titre/secteur) pour respecter la faisabilité :")
        for e in exceptions:
            print(f"   - {e}")
    else:
        print("✅ Filtre ESG appliqué : chaque secteur conserve ≥ 1 titre.")


✅ Filtre ESG appliqué : chaque secteur conserve ≥ 1 titre.


### 7.2) µ ex-ante

In [38]:
import numpy as np
import pandas as pd
import warnings


# 1) Préparation / garanties d'existence
if "work" not in globals():
    work = actions.copy()
if "RIDGE_ALPHA" not in globals():
    RIDGE_ALPHA = 1.0


# Colonnes requises (assumées définies en amont)
# COL_SECTOR, COL_RET1Y, COL_VOL, COL_PE, COL_LTG, COL_UP, COL_ESG


# Petit helper local : z-score par secteur (médiane/MAD avec fallback std)
def _mad(s):
    s = pd.to_numeric(pd.Series(s), errors="coerce")
    s_clean = s[~np.isnan(s)]
    
    # Vérifier qu'il y a assez de données
    if len(s_clean) < 2:
        return np.nan
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        med = np.nanmedian(s)
        if not np.isfinite(med):
            return np.nan
        
        mad = np.nanmedian(np.abs(s - med))
        
        if np.isfinite(mad) and mad > 0:
            return mad
        else:
            std = np.nanstd(s)
            return std if np.isfinite(std) and std > 0 else np.nan


def _zscore_by_sector(df, sector_col, value_col):
    out = pd.Series(index=df.index, dtype=float)
    
    for sect, g in df.groupby(sector_col):
        x = pd.to_numeric(g[value_col], errors="coerce")
        x_clean = x[~np.isnan(x)]
        
        # Vérifier qu'il y a assez de données valides
        if len(x_clean) < 2:
            out.loc[g.index] = np.nan
            continue
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            mu = np.nanmedian(x)
            
            if not np.isfinite(mu):
                out.loc[g.index] = np.nan
                continue
            
            s = _mad(x)
            
            if not np.isfinite(s) or s == 0:
                s = np.nanstd(x)
                if not np.isfinite(s) or s == 0:
                    out.loc[g.index] = np.nan
                    continue
            
            out.loc[g.index] = (x - mu) / s
    
    return out


# 2) (Re)créer les facteurs z-score si absents, puis ZCANDS
if "ZCANDS" not in globals():
    ZCANDS = []


# Momentum ajusté du risque : Return/Vol → z-score sectoriel
if "Z::ret_vol_ratio" not in work.columns and all(c in work.columns for c in [COL_RET1Y, COL_VOL]):
    ret = pd.to_numeric(work[COL_RET1Y], errors="coerce")
    vol = pd.to_numeric(work[COL_VOL], errors="coerce")
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        work["ret_vol_ratio"] = np.where((vol.notna()) & (vol != 0), ret / vol, np.nan)
    
    work["Z::ret_vol_ratio"] = _zscore_by_sector(work, COL_SECTOR, "ret_vol_ratio")


# Value via -P/E
if "Z::neg_pe" not in work.columns and COL_PE in work.columns:
    work["neg_pe"] = -pd.to_numeric(work[COL_PE], errors="coerce")
    work["Z::neg_pe"] = _zscore_by_sector(work, COL_SECTOR, "neg_pe")


# PEG (si LTG>0) puis -PEG
if "Z::neg_peg" not in work.columns and all(c in work.columns for c in [COL_PE, COL_LTG]):
    pe  = pd.to_numeric(work[COL_PE], errors="coerce")
    ltg = pd.to_numeric(work[COL_LTG], errors="coerce")
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        peg = np.where((ltg.notna()) & (ltg > 0), pe / ltg, np.nan)
    
    work["PEG"] = peg
    work["neg_peg"] = -peg
    work["Z::neg_peg"] = _zscore_by_sector(work, COL_SECTOR, "neg_peg")


# Low-Vol = -Vol
if "Z::neg_vol" not in work.columns and COL_VOL in work.columns:
    work["neg_vol"] = -pd.to_numeric(work[COL_VOL], errors="coerce")
    work["Z::neg_vol"] = _zscore_by_sector(work, COL_SECTOR, "neg_vol")


# Z-scores directs disponibles depuis 7.0.1 (si présents), sinon on peut les ignorer
candidates_possible = [
    "Z::ret_vol_ratio", "Z::neg_pe", "Z::neg_peg", "Z::neg_vol",
    f"Z::{COL_LTG}", f"Z::{COL_UP}", f"Z::{COL_ESG}"
]
ZCANDS = [c for c in candidates_possible if c in work.columns]


# 3) µ ex-ante
# (a) Upside pur
mu1 = pd.to_numeric(work.get(COL_UP, pd.Series(index=work.index, dtype=float)), errors="coerce")


# (b) Régression factorielle ridge (par secteur) si Y et X disponibles
mu2 = pd.Series(np.nan, index=work.index, dtype=float)
y_col = COL_RET1Y


if (y_col in work.columns) and ZCANDS:
    for sect, g in work.groupby(COL_SECTOR):
        gidx = g.index
        y = pd.to_numeric(g[y_col], errors="coerce")
        X = g[ZCANDS].astype(float)

        valid = X.notna().all(axis=1) & y.notna()
        n_valid = valid.sum()
        min_samples = max(5, len(ZCANDS) + 1)
        
        if n_valid < min_samples:
            continue

        Xv = X.loc[valid].values
        yv = y.loc[valid].values

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            y_mean = np.nanmean(yv)
            y_std  = np.nanstd(yv)
            
            if not np.isfinite(y_mean) or not np.isfinite(y_std) or y_std == 0:
                y_std = 1.0
            
            yzn = (yv - y_mean) / y_std

        try:
            XtX = Xv.T @ Xv
            beta = np.linalg.solve(XtX + RIDGE_ALPHA * np.eye(XtX.shape[0]), Xv.T @ yzn)
            yhat_z = g[ZCANDS].astype(float).values @ beta
            mu2.loc[gidx] = yhat_z
        except np.linalg.LinAlgError:
            # Matrice singulière, on passe
            continue


# (c) Ensemble 50/50 ; si mu1 ou mu2 manquent, on prend l'autre
mu = pd.Series(np.nan, index=work.index, dtype=float)
has1, has2 = mu1.notna(), mu2.notna()
mu.loc[has1 & has2]  = 0.5 * mu1.loc[has1 & has2] + 0.5 * mu2.loc[has1 & has2]
mu.loc[has1 & ~has2] = mu1.loc[has1 & ~has2]
mu.loc[~has1 & has2] = mu2.loc[~has1 & has2]


# Clipping ex-ante (robustesse)
if mu.notna().any():
    mu_clean = mu[~np.isnan(mu)]
    if len(mu_clean) >= 2:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            lo, hi = np.nanpercentile(mu, [2.5, 97.5])
            if np.isfinite(lo) and np.isfinite(hi):
                mu = mu.clip(lower=lo, upper=hi)


work["mu_ex_ante"] = mu

### 7.3) Sigma (single-index) et bêta proxy

In [39]:
# Vol indice (1 ligne)
vol_index = pd.to_numeric(indice.get(COL_VOL), errors="coerce").squeeze()
vol_index = float(vol_index) if np.isfinite(vol_index) else np.nan

sigma_i = pd.to_numeric(work.get(COL_VOL, pd.Series(index=work.index, dtype=float)), errors="coerce")
beta_i  = sigma_i / vol_index if np.isfinite(vol_index) else np.nan
beta_i  = beta_i.replace([np.inf, -np.inf], np.nan)

# Variance spécifique
spec_var = (sigma_i**2 - (beta_i**2)*(vol_index**2)).clip(lower=0) if np.isfinite(vol_index) else pd.Series(np.nan, index=work.index)

work["beta_proxy"] = beta_i
work["spec_var"]   = spec_var

### 7.4) Pré-sélection Top-K par secteur

In [40]:
# Score composite S = moyenne des z-facteurs disponibles (ZCANDS)
if ZCANDS:
    work["S_composite"] = work[ZCANDS].mean(axis=1, skipna=True)
else:
    # fallback si aucun facteur z n'est dispo: utiliser mu_ex_ante
    work["S_composite"] = work["mu_ex_ante"]

# Nombre de secteurs et K
sectors = work[COL_SECTOR].dropna().unique().tolist()
n_sect  = len(sectors) if len(sectors)>0 else 1
K = max(1, int(np.ceil(N_TARGET / n_sect)))

# Top-K par secteur + Top-1 obligatoires
candidates_idx = []
mandatory_idx  = []
for s, g in work.groupby(COL_SECTOR):
    g = g.sort_values("S_composite", ascending=False)
    take = g.head(K)
    candidates_idx.extend(take.index.tolist())
    if len(g):
        mandatory_idx.append(g.index[0])  # Top-1 obligatoire

candidates_idx = sorted(set(candidates_idx))
mandatory_idx  = sorted(set(mandatory_idx))

cand = work.loc[candidates_idx].copy().reset_index(drop=False).rename(columns={"index":"_row"})
cand["_mandatory"] = cand["_row"].isin(mandatory_idx)


### 7.5) Optimisation Markowitz (SLSQP) — avec contrainte sectorielle ≥ 1 titre/secteur

In [41]:
try:
    from scipy.optimize import minimize
except Exception as e:
    raise ImportError("Cette cellule requiert SciPy (scipy.optimize). Installe-le avec `pip install scipy` puis relance.") from e


# Préparer µ, Sigma, bornes et contraintes (sur l'univers CANDIDAT 'cand')
mu_vec = pd.to_numeric(cand["mu_ex_ante"], errors="coerce").fillna(0).values
sigmas = pd.to_numeric(cand[COL_VOL], errors="coerce").values
betas  = pd.to_numeric(cand["beta_proxy"], errors="coerce").values
spec   = pd.to_numeric(cand["spec_var"], errors="coerce").values


n = len(cand)


# Volatilité de l'indice (déjà calculée dans 7.0.3)
if not (isinstance(vol_index, (float, int)) and np.isfinite(vol_index)):
    # Fallback robuste si vol_index indisponible : diagonale seule
    Sigma = np.diag(np.nan_to_num(sigmas**2, nan=np.nanmedian(sigmas**2)))
else:
    # Modèle single-index : Sigma = beta beta^T * vol_index^2 + diag(spécifique)
    Sigma = np.outer(betas, betas) * (vol_index**2)
    np.fill_diagonal(Sigma, np.diag(Sigma) + np.nan_to_num(spec, nan=0.0))


# Petite régularisation pour SPD numérique
Sigma = Sigma + 1e-8 * np.eye(n)


# Bornes 0..8% (libres, pas de minimum global)
bounds = [(0.0, 0.08) for _ in range(n)]


# 1) Contrainte somme des poids = 1
constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]


# 2) Contrainte sectorielle : somme des poids par secteur >= epsilon_min
#    On définit les masques secteur au NIVEAU DES CANDIDATS (même ordre que w)
epsilon_min = 0.001  # ~0.1% du portefeuille, suffisant pour garantir présence
sector_labels = cand[COL_SECTOR].fillna("Inconnu")
sectors_unique = sector_labels.unique().tolist()


for sect in sectors_unique:
    mask = (sector_labels == sect).astype(float).values  # vecteur binaire taille n
    # inégalité: np.dot(mask, w) - epsilon_min >= 0
    constraints.append({
        "type": "ineq",
        "fun": (lambda w, mask=mask: float(np.dot(mask, w) - epsilon_min))
    })


def portfolio_stats(w):
    er = float(w @ mu_vec)
    var = float(w @ Sigma @ w)
    std = np.sqrt(max(var, 0.0))
    sharpe = er / std if std > 0 else -np.inf
    return er, std, sharpe


def objective_lambda(w, lam):
    # On MINIMISE la fonction opposée : -(w^T mu - lam * w^T Σ w)
    return -(w @ mu_vec - lam * (w @ Sigma @ w))


# Recherche sur grille de lambda (maximisation du Sharpe)
best = {"lam": None, "w": None, "er": -np.inf, "std": np.inf, "sharpe": -np.inf, "res": None}
x0 = np.full(n, 1.0 / n)


for lam in LAMBDA_GRID:
    res = minimize(
        objective_lambda, x0, args=(lam,),
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options=dict(maxiter=1000, ftol=1e-12, disp=False)
    )
    if not res.success:
        continue


    w = res.x
    # Projection numérique douce: respect des bornes et normalisation
    w = np.clip(w, [b[0] for b in bounds], [b[1] for b in bounds])
    s = w.sum()
    if s != 0:
        w = w / s


    # Vérifier les contraintes sectorielles (tolérance numérique)
    ok_sector = True
    for sect in sectors_unique:
        mask = (sector_labels == sect).astype(float).values
        if np.dot(mask, w) < (epsilon_min - 1e-8):
            ok_sector = False
            break
    if not ok_sector:
        continue


    er, std, sh = portfolio_stats(w)
    if sh > best["sharpe"]:
        best.update({"lam": lam, "w": w, "er": er, "std": std, "sharpe": sh, "res": res})
        x0 = w  # warm-start pour accélérer la convergence


w_opt = best["w"]
if w_opt is None:
    raise RuntimeError(
        "Optimisation non résolue sur la grille de lambda avec contraintes sectorielles. "
        "Vérifie que les contraintes sont satisfaisables."
    )


# ===== POST-TRAITEMENT : appliquer 1% minimum aux poids sélectionnés =====
MIN_WEIGHT_SELECTED = 0.01  # 1% minimum pour les actions sélectionnées

# Étape 1 : identifier les actions à inclure (poids > seuil très faible)
threshold_selection = 1e-4  # seuil pour décider si une action est "sélectionnée"
selected_mask = w_opt > threshold_selection

# Étape 2 : re-normaliser en appliquant le minimum aux sélectionnées
w_final = w_opt.copy()

# Forcer 1% minimum pour les actions sélectionnées
w_final[selected_mask] = np.maximum(w_final[selected_mask], MIN_WEIGHT_SELECTED)

# Mettre à 0 les actions non sélectionnées
w_final[~selected_mask] = 0.0

# Étape 3 : re-normaliser pour que la somme = 1
s = w_final.sum()
if s > 0:
    w_final = w_final / s
else:
    w_final = w_opt  # fallback si quelque chose ne va pas


w_opt = w_final


# Résultats DataFrame
port = cand.copy()
port["weight"] = w_opt
port = port[port["weight"] > 1e-8].sort_values("weight", ascending=False).reset_index(drop=True)


# KPIs portefeuille (recalculer avec les nouveaux poids)
ER_p, SD_p, SH_p = portfolio_stats(w_opt)


# Bench proxy (indice) : Sharpe = Return1Y / Vol
ret_idx = float(pd.to_numeric(indice.get(COL_RET1Y), errors="coerce").squeeze())
vol_idx = float(pd.to_numeric(indice.get(COL_VOL), errors="coerce").squeeze())
SH_idx = (ret_idx / vol_idx) if (np.isfinite(ret_idx) and np.isfinite(vol_idx) and vol_idx > 0) else np.nan


print(f"Lambda* sélectionné : {best['lam']:.4g}")
print(f"Portefeuille — E[R]={ER_p:.2f}%, σ={SD_p:.2f}%, Sharpe={SH_p:.2f}  |  Sharpe indice ≈ {SH_idx:.2f}")
print(f"Nombre de positions : {(w_opt > 1e-8).sum()}")


Lambda* sélectionné : 0.01048
Portefeuille — E[R]=9.89%, σ=21.76%, Sharpe=0.45  |  Sharpe indice ≈ 0.60
Nombre de positions : 14


### 7.6) Sorties & diagnostics

In [42]:
# Table finale
cols_show = ["Ticker", COL_SECTOR, "Risk Country", "weight", "mu_ex_ante", COL_VOL, "beta_proxy"]
final_table = port[cols_show].copy()
final_table = final_table.rename(columns={
    "weight":"Poids",
    "mu_ex_ante":"µ ex-ante",
    COL_VOL:"Vol 1Y",
    "beta_proxy":"Beta (proxy)",
})
# Formats
for c in ["Poids","µ ex-ante","Vol 1Y"]:
    if c in final_table.columns:
        final_table[c] = final_table[c].map(lambda x: f"{x:.2f} %")
if "Beta (proxy)" in final_table.columns:
    final_table["Beta (proxy)"] = port["beta_proxy"].map(lambda x: f"{x:.2f}" if pd.notna(x) else "—")
if "Score (Z-moy)" in final_table.columns:
    final_table["Score (Z-moy)"] = port["S_composite"].map(lambda x: f"{x:.2f}" if pd.notna(x) else "—")

display(final_table)

# Agrégat sectoriel des poids
weights_by_sector = port.groupby(COL_SECTOR, as_index=False)["weight"].sum().sort_values("weight", ascending=False)
weights_by_sector["Poids (%)"] = weights_by_sector["weight"]*100

print("\n=== KPIs Portefeuille ===")
print(f"E[R] (ex-ante) : {ER_p:.2f} %")
print(f"Vol (ex-ante)  : {SD_p:.2f} %")
print(f"Sharpe (ex-ante): {SH_p:.2f}")
print(f"Sharpe indice ≈ : {SH_idx:.2f}")

# Graphiques (Plotly)
import plotly.express as px
import plotly.graph_objects as go

# 1) Bar — poids par titre (Top 25)
topN = min(25, len(port))
fig_w_bar = px.bar(
    port.head(topN), x="Ticker", y="weight",
    color_discrete_sequence=[COLORS["primary"]],
    hover_data={"weight":":.4f", "mu_ex_ante":":.2f", COL_VOL:":.2f", "beta_proxy":":.2f"},
    template="plotly_white"
)
fig_w_bar.update_yaxes(title="Poids", tickformat=".0%")
fig_w_bar.update_xaxes(title="Ticker")
fig_w_bar = apply_plotly_style(fig_w_bar, title="Portefeuille — Poids par titre (Top 25)")
fig_w_bar.update_layout(height=480, margin=dict(l=0,r=0,t=60,b=80))
fig_w_bar.show()

# 2) Donut — répartition sectorielle
fig_sector = px.pie(
    weights_by_sector, names=COL_SECTOR, values="weight",
    hole=0.55, template="plotly_white",
    color_discrete_sequence=px.colors.sequential.Blues
)
fig_sector.update_traces(textposition="inside", texttemplate="%{label}<br>%{percent:.0%}", hovertemplate="%{label}<br>%{value:.2%}<extra></extra>")
fig_sector = apply_plotly_style(fig_sector, title="Répartition sectorielle du portefeuille")
fig_sector.update_layout(height=460, margin=dict(l=0,r=0,t=60,b=40))
fig_sector.show()

# 3) Point — µ vs contribution au risque approximative
# --- Contributions au risque alignées sur les titres retenus ---
Sigma_w = Sigma @ w_opt
tot_var = float(w_opt @ Sigma_w)
cr_full = (w_opt * Sigma_w) / tot_var if tot_var > 0 else np.zeros_like(w_opt)

# Prépare une table d'alignement "cand" -> CRisk sur la clé '_row'
cr_map = cand[["_row"]].copy()
cr_map["CRisk"] = cr_full

# Merge pour rattacher la CRisk uniquement aux titres retenus (port)
port = port.merge(cr_map, on="_row", how="left")

# DataFrame de diagnostic (aligné)
diag_df = pd.DataFrame({
    "Ticker": port["Ticker"],
    "Poids": port["weight"],
    "mu": pd.to_numeric(port["mu_ex_ante"], errors="coerce"),
    "CRisk": port["CRisk"]
})

# Scatter diagnostic µ vs contribution au risque
fig_diag = px.scatter(
    diag_df, x="mu", y="CRisk", size="Poids",
    hover_name="Ticker", template="plotly_white",
    color_discrete_sequence=[COLORS["secondary"]]
)
fig_diag.update_layout(
    xaxis_title="µ ex-ante (%)",
    yaxis_title="Contribution au risque (part de variance)",
    height=460, margin=dict(l=0, r=0, t=60, b=40)
)
fig_diag = apply_plotly_style(fig_diag, title="Diagnostic — µ vs contribution au risque")
fig_diag.show()



,Ticker,Sector (1),Risk Country,Poids,µ ex-ante,Vol 1Y,Beta (proxy)
0,TTE FP Equity,Energy,FRANCE,0.08 %,18.26 %,20.98 %,1.30
1,AI FP Equity,Materials,FRANCE,0.08 %,4.77 %,19.55 %,1.21
2,ENEL IM Equity,Utilities,ITALY,0.08 %,4.60 %,18.47 %,1.14
3,DSY FP Equity,Information Technology,FRANCE,0.08 %,13.16 %,27.14 %,1.68
4,URW FP Equity,Real Estate,FRANCE,0.08 %,15.16 %,25.19 %,1.56
5,SAN FP Equity,Health Care,USA,0.08 %,20.10 %,20.22 %,1.25
6,ORA FP Equity,Communication Services,FRANCE,0.08 %,7.35 %,15.72 %,0.97
7,PUB FP Equity,Communication Services,FRANCE,0.08 %,13.12 %,23.38 %,1.44
8,PRX NA Equity,Consumer Discretionary,CHINA,0.08 %,17.24 %,33.11 %,2.04
9,CS FP Equity,Financials,FRANCE,0.08 %,4.37 %,19.76 %,1.22



=== KPIs Portefeuille ===
E[R] (ex-ante) : 9.89 %
Vol (ex-ante)  : 21.76 %
Sharpe (ex-ante): 0.45
Sharpe indice ≈ : 0.60


### 7.7) Simulation de Monte Carlo

In [43]:


# Prérequis depuis 7.0.6/7.0.7 : port, cand, mu_vec (en % annuels), Sigma (en %^2 annuels), COLORS, apply_plotly_style

# -----------------------
# Paramètres
# -----------------------
N_SIMS_PATHS = 500      # nombre de trajectoires simulées
N_SHOW       = 50       # nombre de trajectoires affichées (pour la lisibilité)
N_DAYS       = 252
SEED         = 123
USE_GEOMETRIC = True     # log-returns (GBM approx)
V0           = 100.0     # valeur initiale (base 100)

np.random.seed(SEED)

# -----------------------
# Alignement des inputs (comme 7.0.7)
# -----------------------
sel_rows = port["_row"].tolist() if "_row" in port.columns else cand.loc[port.index, "_row"].tolist()
idx_map  = {row: i for i, row in enumerate(cand["_row"])}
idx_sel  = np.array([idx_map[r] for r in sel_rows], dtype=int)

w        = port["weight"].values.astype(float)                    # (k,)
muA_pc   = mu_vec[idx_sel].astype(float)                          # µ annuel en %
SA_pc2   = Sigma[np.ix_(idx_sel, idx_sel)].astype(float)         # Σ annuel en %^2

# % -> décimaux
muA = muA_pc / 100.0
SA  = SA_pc2 / (100.0**2)

# Journalier
mu_d = muA / N_DAYS
S_d  = SA  / N_DAYS

# Projection PSD pour stabilité
eigval, eigvec = np.linalg.eigh(S_d)
eigval = np.clip(eigval, a_min=1e-12, a_max=None)
S_d_psd = (eigvec @ np.diag(eigval) @ eigvec.T).astype(float)

k = len(w)
if k == 0:
    raise RuntimeError("Le portefeuille ne contient aucun titre (k=0).")

# -----------------------
# Simulation des chemins
# -----------------------
# Tenseur tirages : (days, sims, k)
Z = np.random.multivariate_normal(mean=np.zeros(k), cov=S_d_psd, size=(N_DAYS, N_SIMS_PATHS))

if USE_GEOMETRIC:
    drift_adj = (mu_d - 0.5 * np.diag(S_d_psd))  # (k,)
    Rlog = Z + drift_adj                         # (days, sims, k)
    # log-retour quotidien du portefeuille
    rp_log_daily = np.tensordot(Rlog, w, axes=([2],[0]))  # (days, sims)
    # valeur cumulée : V_t = V0 * exp(cumsum(rp_log_daily))
    cum_log = rp_log_daily.cumsum(axis=0)
    V_paths = V0 * np.exp(cum_log)                          # (days, sims)
else:
    R = Z + mu_d                                           # (days, sims, k)
    rp_daily = np.tensordot(R, w, axes=([2],[0]))          # (days, sims)
    V_paths = V0 * (1.0 + rp_daily).cumprod(axis=0)        # (days, sims)

# -----------------------
# Fan chart (médiane & bandes)
# -----------------------
t = np.arange(1, N_DAYS+1)
p50 = np.percentile(V_paths, 50, axis=1)
p05 = np.percentile(V_paths, 5,  axis=1)
p95 = np.percentile(V_paths, 95, axis=1)

fig = go.Figure()

# Bande 5–95 %
fig.add_trace(go.Scatter(
    x=t, y=p95, line=dict(width=0), showlegend=False, hoverinfo="skip"
))
fig.add_trace(go.Scatter(
    x=t, y=p05, line=dict(width=0), fill='tonexty', fillcolor='rgba(8,27,72,0.08)',
    name='Bandes 5–95 %', line_color='rgba(0,0,0,0)', hoverinfo="skip"
))

# Médiane
fig.add_trace(go.Scatter(
    x=t, y=p50, mode='lines',
    line=dict(color=COLORS["primary"], width=2),
    name='Médiane'
))

# -----------------------
# Trajectoires individuelles (échantillon)
# -----------------------
to_plot = min(N_SHOW, N_SIMS_PATHS)
for j in range(to_plot):
    fig.add_trace(go.Scatter(
        x=t, y=V_paths[:, j],
        mode='lines',
        line=dict(color='rgba(81,156,221,0.35)', width=1),
        showlegend=False
    ))

fig.update_layout(
    xaxis_title="Jours ouvrés",
    yaxis_title="Valeur du portefeuille (base 100)",
    margin=dict(l=0, r=0, t=60, b=40),
    height=520
)
fig = apply_plotly_style(fig, title="Monte Carlo - Trajectoires d'évolution du portefeuille (base 100)")
fig.show()
